Tyler Malka

CSCI E-82 Advanced Machine Learning

Final Project: Self-Supervised Attention for Super-Resolution

Main notebook

Now that we're familiar with implementing various SR techniques, we're going to set up the official RCAN implementation as our baseline and attempt to outperform it. (See preliminary notebook for initial testing of different SR methods)

In [ ]:
# ChatGPT assisted with code generation

In [ ]:
import os
import torch
import torch.nn as nn
from collections import OrderedDict
import sys
import matplotlib.pyplot as plt
from matplotlib import figure
from google.colab import drive

def setup_rcan():
    """Setup RCAN repository and models"""
    # Mount Google Drive if not already mounted
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    # Clone RCAN if not exists
    if not os.path.exists('RCAN'):
        !git clone https://github.com/yulunzhang/RCAN.git

    # Add RCAN to path
    if 'RCAN' not in sys.path:
        sys.path.append('./RCAN/RCAN_TrainCode/code')

    # Import necessary modules from RCAN
    from model.rcan import RCAN
    return RCAN

class RCANFeatureExtractor(nn.Module):
    """Wrapper for RCAN to extract intermediate features"""
    def __init__(self, rcan_model):
        super().__init__()
        self.head = rcan_model.head
        self.body = rcan_model.body
        self.tail = rcan_model.tail

    def extract_features(self, x):
        """Extract features before the final upsampling"""
        x = self.head(x)
        body_output = self.body(x)
        # Store the input for residual connection
        return x, body_output

    def complete_sr(self, features):
        """Complete super-resolution with extracted features"""
        input_feat, body_output = features
        # Add the residual connection
        x = body_output + input_feat
        # Apply final upsampling
        x = self.tail(x)
        return x

    def forward(self, x):
        """Forward pass through the entire model"""
        features = self.extract_features(x)
        return self.complete_sr(features)

def load_rcan_model(model_path='/content/drive/MyDrive/E82/finalproject/RCAN_BIX4.pt'):
    """Load pretrained RCAN model"""
    # Import RCAN
    RCAN = setup_rcan()

    # Complete args dictionary with all required parameters
    args = {
        'n_resgroups': 10,
        'n_resblocks': 20,
        'n_feats': 64,
        'scale': [4],
        'rgb_range': 255,
        'n_colors': 3,
        'res_scale': 1,
        'reduction': 16,
        'conv': nn.Conv2d,
        'kernel_size': 3,
        'act': nn.ReLU(True),
        'precision': 'single',
        'cpu': False,
    }

    # Convert args dictionary to object-like structure
    class Args:
        def __init__(self, **entries):
            self.__dict__.update(entries)

    args = Args(**args)

    # Create model
    print("Creating RCAN model...")
    model = RCAN(args)

    # Load pretrained weights
    print(f"Loading weights from {model_path}")
    state_dict = torch.load(model_path, weights_only=True)  # Added weights_only=True
    model.load_state_dict(state_dict)
    print("Weights loaded successfully")

    # Wrap model for feature extraction
    model = RCANFeatureExtractor(model)
    model.eval()

    return model

if __name__ == "__main__":
    # Test setup
    try:
        print("Setting up RCAN model...")
        model = load_rcan_model()
        print("RCAN model loaded successfully")

        # Test with dummy input
        x = torch.randn(1, 3, 32, 32)
        with torch.no_grad():
            features = model.extract_features(x)
            print(f"Input feature shape: {features[0].shape}")
            print(f"Body output shape: {features[1].shape}")

            output = model.complete_sr(features)
            print(f"Final output shape: {output.shape}")

    except Exception as e:
        print(f"Error during setup: {str(e)}")
        import traceback
        print(traceback.format_exc())

Input features are being transformed to 64 channels: [1, 64, 32, 32]

Body maintains the same dimensions: [1, 64, 32, 32]

Final output is correctly upscaled 4x with 3 channels: [1, 3, 128, 128]

Correctly scaling to 4x, so we can continue.

In [ ]:
import os
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
import glob
from tqdm import tqdm
from google.colab import drive

class SRDataset(Dataset):
    def __init__(self, root_dir, scale=4, patch_size=96, train=True):
        """
        Args:
            root_dir (str): Directory with images (e.g., Set5 or Set14 folder)
            scale (int): Super resolution scale factor
            patch_size (int): Size of HR patches to extract
            train (bool): If True, prepare for training (random crops), else for evaluation
        """
        self.scale = scale
        self.patch_size = patch_size
        self.train = train

        # Mount Google Drive if not already mounted
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')

        # Find all images in the directory
        self.image_files = sorted(glob.glob(os.path.join(root_dir, '*.png')))
        if len(self.image_files) == 0:
            raise RuntimeError(f"No PNG images found in {root_dir}")

        print(f"Found {len(self.image_files)} images in {root_dir}")

        # Basic augmentations for training
        self.augment = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip()
        ]) if train else None

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load HR image
        img_path = self.image_files[idx]
        hr_image = Image.open(img_path).convert('RGB')

        # Handle training vs evaluation
        if self.train:
            # Random crop for training
            i, j, h, w = transforms.RandomCrop.get_params(
                hr_image, output_size=(self.patch_size, self.patch_size))
            hr_image = TF.crop(hr_image, i, j, h, w)

            # Apply augmentations
            if self.augment:
                hr_image = self.augment(hr_image)
        else:
            # For validation, ensure dimensions are divisible by scale
            w, h = hr_image.size
            w = w - w % self.scale
            h = h - h % self.scale
            hr_image = hr_image.crop((0, 0, w, h))

        # Convert to tensor
        hr_tensor = TF.to_tensor(hr_image)

        # Create LR image using bicubic downsampling
        lr_tensor = TF.resize(hr_tensor,
                            size=[s // self.scale for s in hr_tensor.shape[-2:]],
                            interpolation=TF.InterpolationMode.BICUBIC)

        return lr_tensor, hr_tensor

def setup_datasets(batch_size=16):
    """Setup training and validation dataloaders"""
    # Paths to your datasets
    set5_path = '/content/drive/MyDrive/E82/finalproject/Set5'
    set14_path = '/content/drive/MyDrive/E82/finalproject/Set14'

    # Create datasets
    set5_dataset = SRDataset(root_dir=set5_path, scale=4, patch_size=96, train=True)
    set14_dataset = SRDataset(root_dir=set14_path, scale=4, patch_size=96, train=False)

    # Create dataloaders
    train_loader = DataLoader(
        set5_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    val_loader = DataLoader(
        set14_dataset,
        batch_size=1,  # Evaluate one image at a time
        shuffle=False,
        num_workers=1,
        pin_memory=True
    )

    return train_loader, val_loader

if __name__ == "__main__":
    # Test the dataset setup
    try:
        print("Setting up datasets...")
        train_loader, val_loader = setup_datasets(batch_size=4)

        print(f"\nNumber of training batches: {len(train_loader)}")
        print(f"Number of validation images: {len(val_loader)}")

        # Test a batch
        lr_batch, hr_batch = next(iter(train_loader))
        print(f"\nSample batch shapes:")
        print(f"LR batch: {lr_batch.shape}")
        print(f"HR batch: {hr_batch.shape}")

        # Verify scale factor
        lr_h, lr_w = lr_batch.shape[-2:]
        hr_h, hr_w = hr_batch.shape[-2:]
        print(f"\nScale factor verification:")
        print(f"Height scale: {hr_h/lr_h:.0f}x")
        print(f"Width scale: {hr_w/lr_w:.0f}x")

    except Exception as e:
        print(f"Error during setup: {str(e)}")
        import traceback
        print(traceback.format_exc())

Everything is working properly. Let's build our attention mechanism next.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

class SelfSupervisedAttention(nn.Module):
    """Self-supervised auxiliary network for dynamic pixel importance prediction"""
    def __init__(self, in_channels=64):
        super().__init__()
        self.in_channels = in_channels

        # Spatial feature extraction
        self.conv1 = nn.Conv2d(in_channels, in_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv2 = nn.Conv2d(in_channels, in_channels, 3, padding=1)  # Keep same channels
        self.bn2 = nn.BatchNorm2d(in_channels)

        # Attention prediction
        self.conv3 = nn.Conv2d(in_channels, in_channels//2, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(in_channels//2)
        self.conv4 = nn.Conv2d(in_channels//2, 1, 1)

        # Channel attention
        self.spatial_pool = nn.AdaptiveAvgPool2d(1)
        self.channel_attention = nn.Sequential(
            nn.Linear(in_channels, in_channels//4),
            nn.ReLU(True),
            nn.Linear(in_channels//4, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Initial feature extraction
        feat = F.relu(self.bn1(self.conv1(x)))
        feat = F.relu(self.bn2(self.conv2(feat)))  # [B, 64, H, W]

        # Channel attention
        channel_att = self.spatial_pool(x).squeeze(-1).squeeze(-1)  # [B, 64]
        channel_att = self.channel_attention(channel_att)  # [B, 64]
        channel_att = channel_att.view(-1, self.in_channels, 1, 1)  # [B, 64, 1, 1]

        # Apply channel attention
        feat = feat * channel_att  # [B, 64, H, W]

        # Final attention prediction
        feat = F.relu(self.bn3(self.conv3(feat)))  # [B, 32, H, W]
        attention = torch.sigmoid(self.conv4(feat))  # [B, 1, H, W]

        return attention

class AttentionAugmentedRCAN(nn.Module):
    def __init__(self, base_rcan, freeze_base=True):
        super().__init__()
        self.rcan = base_rcan
        self.attention_net = SelfSupervisedAttention(64)  # RCAN uses 64 channels

        if freeze_base:
            for param in self.rcan.parameters():
                param.requires_grad = False

    def forward(self, x, mode='inference'):
        if mode == 'pre_training':
            # Extract features but don't apply attention yet
            feats = self.rcan.extract_features(x)
            attention = self.attention_net(feats[1])  # Use body output for attention
            return attention

        # Normal forward pass with attention
        input_feat, body_feat = self.rcan.extract_features(x)
        attention = self.attention_net(body_feat)
        weighted_feat = body_feat * attention

        # Complete super-resolution
        sr_output = self.rcan.complete_sr((input_feat, weighted_feat))

        return sr_output, attention

def entropy_loss(attention_maps):
    """Entropy-based regularization for attention maps"""
    eps = 1e-8
    entropy = -(attention_maps * torch.log(attention_maps + eps))
    return entropy.mean()

def spatial_consistency_loss(attention_maps):
    """Spatial consistency loss for attention maps"""
    horizontal = F.l1_loss(attention_maps[..., :, 1:], attention_maps[..., :, :-1])
    vertical = F.l1_loss(attention_maps[..., 1:, :], attention_maps[..., :-1, :])
    return horizontal + vertical

def get_reconstruction_difficulty(sr_output, hr_target):
    """Calculate pixel-wise reconstruction difficulty"""
    with torch.no_grad():
        diff = torch.abs(sr_output - hr_target)
        # Normalize to [0, 1] range
        diff = (diff - diff.min()) / (diff.max() - diff.min() + 1e-8)
        return diff.mean(dim=1, keepdim=True)  # Average across channels

class Trainer:
    def __init__(self, model, train_loader, val_loader, device):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.writer = SummaryWriter('runs/attention_rcan')

    def pre_training_step(self, batch, optimizer):
        """Pre-training step for attention network"""
        lr_imgs, hr_imgs = [x.to(self.device) for x in batch]
        optimizer.zero_grad()

        # Get base RCAN output for difficulty estimation
        with torch.no_grad():
            sr_output = self.model.rcan(lr_imgs)
            difficulty = get_reconstruction_difficulty(sr_output, hr_imgs)

        # Train attention network
        attention = self.model(lr_imgs, mode='pre_training')

        # Losses
        prediction_loss = F.mse_loss(attention, difficulty)
        entropy_reg = entropy_loss(attention)
        consistency = spatial_consistency_loss(attention)

        total_loss = prediction_loss + 0.1 * entropy_reg + 0.1 * consistency

        total_loss.backward()
        optimizer.step()

        return {
            'prediction_loss': prediction_loss.item(),
            'entropy_loss': entropy_reg.item(),
            'consistency_loss': consistency.item(),
            'total_loss': total_loss.item()
        }

    def fine_tuning_step(self, batch, optimizer):
        """Fine-tuning step for joint training"""
        lr_imgs, hr_imgs = [x.to(self.device) for x in batch]
        optimizer.zero_grad()

        # Forward pass with attention
        sr_output, attention = self.model(lr_imgs)

        # Calculate losses
        reconstruction_loss = F.l1_loss(sr_output, hr_imgs)
        entropy_reg = entropy_loss(attention)
        consistency = spatial_consistency_loss(attention)

        # Get current reconstruction difficulty
        difficulty = get_reconstruction_difficulty(sr_output, hr_imgs)
        attention_guidance = F.mse_loss(attention, difficulty.detach())

        total_loss = reconstruction_loss + 0.1 * entropy_reg + 0.1 * consistency + 0.1 * attention_guidance

        total_loss.backward()
        optimizer.step()

        return {
            'reconstruction_loss': reconstruction_loss.item(),
            'entropy_loss': entropy_reg.item(),
            'consistency_loss': consistency.item(),
            'attention_guidance': attention_guidance.item(),
            'total_loss': total_loss.item()
        }

if __name__ == "__main__":
    # Quick test of attention mechanism
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    x = torch.randn(4, 64, 32, 32).to(device)
    attention_net = SelfSupervisedAttention().to(device)
    attention_maps = attention_net(x)
    print(f"Input shape: {x.shape}")
    print(f"Attention map shape: {attention_maps.shape}")

Input: [4, 64, 32, 32] - batch of 4, 64 channels from RCAN features

Output: [4, 1, 32, 32] - same spatial dimensions but single channel attention map

Everything is working properly.

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import time
import os
import torch.nn.functional as F

# Use models and classes we defined above
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class Trainer:
    def __init__(self, model, train_loader, val_loader, device):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.writer = SummaryWriter('runs/attention_rcan')

    def pre_training_step(self, batch, optimizer):
        """Pre-training step for attention network"""
        lr_imgs, hr_imgs = [x.to(self.device) for x in batch]
        optimizer.zero_grad()

        # Get base RCAN output for difficulty estimation
        with torch.no_grad():
            sr_output = self.model.rcan(lr_imgs)
            difficulty = get_reconstruction_difficulty(sr_output, hr_imgs)  # [B, 1, H, H]

        # Train attention network
        attention = self.model(lr_imgs, mode='pre_training')  # [B, 1, h, h]

        # Upscale attention to match HR resolution
        attention_upscaled = F.interpolate(
            attention,
            size=difficulty.shape[-2:],
            mode='bilinear',
            align_corners=False
        )

        # Losses
        prediction_loss = F.mse_loss(attention_upscaled, difficulty)
        entropy_reg = entropy_loss(attention)  # Keep entropy loss on LR attention
        consistency = spatial_consistency_loss(attention)  # Keep consistency on LR attention

        total_loss = prediction_loss + 0.1 * entropy_reg + 0.1 * consistency

        total_loss.backward()
        optimizer.step()

        return {
            'prediction_loss': prediction_loss.item(),
            'entropy_loss': entropy_reg.item(),
            'consistency_loss': consistency.item(),
            'total_loss': total_loss.item()
        }

    def fine_tuning_step(self, batch, optimizer):
        """Fine-tuning step for joint training"""
        lr_imgs, hr_imgs = [x.to(self.device) for x in batch]
        optimizer.zero_grad()

        # Forward pass with attention
        sr_output, attention = self.model(lr_imgs)

        # Calculate losses
        reconstruction_loss = F.l1_loss(sr_output, hr_imgs)
        entropy_reg = entropy_loss(attention)
        consistency = spatial_consistency_loss(attention)

        # Get current reconstruction difficulty
        difficulty = get_reconstruction_difficulty(sr_output, hr_imgs)

        # Upscale attention to match HR resolution
        attention_upscaled = F.interpolate(
            attention,
            size=difficulty.shape[-2:],
            mode='bilinear',
            align_corners=False
        )
        attention_guidance = F.mse_loss(attention_upscaled, difficulty.detach())

        total_loss = reconstruction_loss + 0.1 * entropy_reg + 0.1 * consistency + 0.1 * attention_guidance

        total_loss.backward()
        optimizer.step()

        return {
            'reconstruction_loss': reconstruction_loss.item(),
            'entropy_loss': entropy_reg.item(),
            'consistency_loss': consistency.item(),
            'attention_guidance': attention_guidance.item(),
            'total_loss': total_loss.item()
        }

# Load RCAN model
print("Loading base RCAN model...")
base_model = load_rcan_model()
base_model = base_model.to(device)

# Create augmented model
print("Creating attention-augmented model...")
model = AttentionAugmentedRCAN(base_model, freeze_base=True)
model = model.to(device)

# Setup data
print("Setting up datasets...")
train_loader, val_loader = setup_datasets(batch_size=4)

# Create trainer
trainer = Trainer(model, train_loader, val_loader, device)

# Training configuration
config = {
    'pre_train_epochs': 10,
    'fine_tune_epochs': 20,
    'pre_train_lr': 1e-4,
    'fine_tune_lr': 5e-5
}

# Stage 1: Pre-training attention network
print("\nStage 1: Pre-training attention network")
attention_optimizer = torch.optim.Adam(
    model.attention_net.parameters(),
    lr=config['pre_train_lr']
)

for epoch in range(config['pre_train_epochs']):
    model.train()

    with tqdm(train_loader, desc=f'Pre-training Epoch {epoch+1}/{config["pre_train_epochs"]}') as pbar:
        for lr_imgs, hr_imgs in pbar:
            losses = trainer.pre_training_step((lr_imgs, hr_imgs), attention_optimizer)
            pbar.set_postfix({k: f'{v:.4f}' for k, v in losses.items()})

    # Validation
    if (epoch + 1) % 2 == 0:
        print(f"\nValidating epoch {epoch+1}...")
        model.eval()
        val_losses = []
        with torch.no_grad():
            for lr_imgs, hr_imgs in val_loader:
                lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
                attention = model(lr_imgs, mode='pre_training')
                # Log first batch attention maps
                if len(val_losses) == 0:
                    print(f"Sample attention map range: {attention.min().item():.4f} to {attention.max().item():.4f}")

# Stage 2: Fine-tuning
print("\nStage 2: Joint fine-tuning")
# Unfreeze RCAN
for param in model.rcan.parameters():
    param.requires_grad = True

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=config['fine_tune_lr']
)

best_psnr = 0
for epoch in range(config['fine_tune_epochs']):
    model.train()

    with tqdm(train_loader, desc=f'Fine-tuning Epoch {epoch+1}/{config["fine_tune_epochs"]}') as pbar:
        for lr_imgs, hr_imgs in pbar:
            losses = trainer.fine_tuning_step((lr_imgs, hr_imgs), optimizer)
            pbar.set_postfix({k: f'{v:.4f}' for k, v in losses.items()})

    # Validation
    if (epoch + 1) % 2 == 0:
        print(f"\nValidating epoch {epoch+1}...")
        model.eval()
        psnr_values = []
        with torch.no_grad():
            for lr_imgs, hr_imgs in val_loader:
                lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
                sr_output, attention = model(lr_imgs)

                # Calculate PSNR
                mse = torch.mean((sr_output - hr_imgs) ** 2)
                psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))
                psnr_values.append(psnr.item())

        avg_psnr = sum(psnr_values) / len(psnr_values)
        print(f"Average PSNR: {avg_psnr:.2f}")

        # Save best model
        if avg_psnr > best_psnr:
            best_psnr = avg_psnr
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'psnr': best_psnr,
            }, 'best_model.pt')
            print(f"Saved new best model with PSNR {best_psnr:.2f}")

print("Training completed!")
print(f"Best PSNR achieved: {best_psnr:.2f}")

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
from PIL import Image
import glob
from google.colab import drive
import matplotlib.pyplot as plt
import gdown

class SRDataset(Dataset):
    def __init__(self, root_dir, scale=4, patch_size=96, train=True):
        self.scale = scale
        self.patch_size = patch_size
        self.train = train

        # Mount Google Drive if not already mounted
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')

        # Find all images in the directory
        self.image_files = sorted(glob.glob(os.path.join(root_dir, '*.png')))
        if len(self.image_files) == 0:
            raise RuntimeError(f"No PNG images found in {root_dir}")

        print(f"Found {len(self.image_files)} images in {root_dir}")

        # Basic augmentations for training
        self.augment = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip()
        ]) if train else None

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load HR image
        img_path = self.image_files[idx]
        hr_image = Image.open(img_path).convert('RGB')

        # Handle training vs evaluation
        if self.train:
            # Random crop for training
            i, j, h, w = transforms.RandomCrop.get_params(
                hr_image, output_size=(self.patch_size, self.patch_size))
            hr_image = TF.crop(hr_image, i, j, h, w)

            # Apply augmentations
            if self.augment:
                hr_image = self.augment(hr_image)
        else:
            # For validation, ensure dimensions are divisible by scale
            w, h = hr_image.size
            w = w - w % self.scale
            h = h - h % self.scale
            hr_image = hr_image.crop((0, 0, w, h))

        # Convert to tensor
        hr_tensor = TF.to_tensor(hr_image)

        # Create LR image using bicubic downsampling
        lr_tensor = TF.resize(hr_tensor,
                            size=[s // self.scale for s in hr_tensor.shape[-2:]],
                            interpolation=TF.InterpolationMode.BICUBIC)

        return lr_tensor, hr_tensor

def download_div2k():
    """Download DIV2K dataset if not present"""
    # DIV2K training data
    train_url = "http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip"
    if not os.path.exists('DIV2K_train_HR'):
        print("Downloading DIV2K training data...")
        gdown.download(train_url, 'DIV2K_train_HR.zip', quiet=False)
        !unzip -q DIV2K_train_HR.zip
        !rm DIV2K_train_HR.zip

def setup_datasets(batch_size=16):
    """Setup training and validation dataloaders"""
    # Download DIV2K if needed
    download_div2k()

    # Create datasets
    train_dataset = SRDataset(
        root_dir='DIV2K_train_HR',
        scale=4,
        patch_size=96,
        train=True
    )

    val_dataset = SRDataset(
        root_dir='/content/drive/MyDrive/E82/finalproject/Set14',
        scale=4,
        patch_size=96,
        train=False
    )

    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=1,
        pin_memory=True
    )

    return train_loader, val_loader

def visualize_results(model, val_loader, device, save_path=None):
    """Visualize and compare results between bicubic, RCAN, and our model"""
    model.eval()
    with torch.no_grad():
        # Get first validation image
        lr_img, hr_img = next(iter(val_loader))
        lr_img, hr_img = lr_img.to(device), hr_img.to(device)

        # Bicubic upscaling
        bicubic = F.interpolate(
            lr_img,
            scale_factor=4,
            mode='bicubic',
            align_corners=False
        )

        # Get RCAN output
        rcan_output = model.rcan(lr_img)

        # Get our model output
        sr_output, attention = model(lr_img)

        # Convert to images for plotting
        def tensor_to_image(x):
            x = x.cpu().squeeze(0).permute(1, 2, 0).clamp(0, 1).numpy()
            return x

        # Create figure
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))

        # Plot images
        axes[0, 0].imshow(tensor_to_image(lr_img))
        axes[0, 0].set_title('LR Input')
        axes[0, 1].imshow(tensor_to_image(bicubic))
        axes[0, 1].set_title('Bicubic')
        axes[0, 2].imshow(tensor_to_image(hr_img))
        axes[0, 2].set_title('HR Ground Truth')
        axes[1, 0].imshow(tensor_to_image(rcan_output))
        axes[1, 0].set_title('RCAN Output')
        axes[1, 1].imshow(tensor_to_image(sr_output))
        axes[1, 1].set_title('Our Model Output')
        axes[1, 2].imshow(tensor_to_image(attention.repeat(1, 3, 1, 1)))
        axes[1, 2].set_title('Attention Map')

        # Remove axes
        for ax in axes.flat:
            ax.axis('off')

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
        plt.show()

if __name__ == "__main__":
    # Test the dataset setup
    try:
        print("Setting up datasets...")
        train_loader, val_loader = setup_datasets(batch_size=16)

        print(f"\nNumber of training batches: {len(train_loader)}")
        print(f"Number of validation images: {len(val_loader)}")

        # Test a batch
        lr_batch, hr_batch = next(iter(train_loader))
        print(f"\nSample batch shapes:")
        print(f"LR batch: {lr_batch.shape}")
        print(f"HR batch: {hr_batch.shape}")

    except Exception as e:
        print(f"Error during setup: {str(e)}")
        import traceback
        print(traceback.format_exc())

In [ ]:
# Load best model
checkpoint = torch.load('best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])

# Visualize results
visualize_results(model, val_loader, device, save_path='comparison_before_div2k.png')

In [ ]:
# Use models and classes we defined above
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load RCAN model
print("Loading base RCAN model...")
base_model = load_rcan_model()
base_model = base_model.to(device)

# Create augmented model
print("Creating attention-augmented model...")
model = AttentionAugmentedRCAN(base_model, freeze_base=True)
model = model.to(device)

# Setup data
print("Setting up datasets...")
train_loader, val_loader = setup_datasets(batch_size=16)

# Create trainer
trainer = Trainer(model, train_loader, val_loader, device)

# Training configuration
config = {
    'pre_train_epochs': 50,
    'fine_tune_epochs': 100,
    'pre_train_lr': 1e-4,
    'fine_tune_lr': 1e-5,
    'val_frequency': 5  # Validate every 5 epochs to save time
}

# Stage 1: Pre-training attention network
print("\nStage 1: Pre-training attention network")
attention_optimizer = torch.optim.Adam(
    model.attention_net.parameters(),
    lr=config['pre_train_lr']
)

for epoch in range(config['pre_train_epochs']):
    model.train()
    epoch_losses = []

    with tqdm(train_loader, desc=f'Pre-training Epoch {epoch+1}/{config["pre_train_epochs"]}') as pbar:
        for lr_imgs, hr_imgs in pbar:
            losses = trainer.pre_training_step((lr_imgs, hr_imgs), attention_optimizer)
            epoch_losses.append(losses)
            pbar.set_postfix({k: f'{v:.4f}' for k, v in losses.items()})

    # Calculate average losses for epoch
    avg_losses = {k: sum(d[k] for d in epoch_losses) / len(epoch_losses)
                 for k in epoch_losses[0].keys()}

    # Validation
    if (epoch + 1) % config['val_frequency'] == 0:
        print(f"\nEpoch {epoch+1} average losses:")
        for k, v in avg_losses.items():
            print(f"{k}: {v:.4f}")

        print(f"\nValidating epoch {epoch+1}...")
        model.eval()
        val_losses = []
        with torch.no_grad():
            for lr_imgs, hr_imgs in val_loader:
                lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
                attention = model(lr_imgs, mode='pre_training')
                if len(val_losses) == 0:
                    print(f"Sample attention map range: {attention.min().item():.4f} to {attention.max().item():.4f}")

# Stage 2: Fine-tuning
print("\nStage 2: Joint fine-tuning")
# Unfreeze RCAN
for param in model.rcan.parameters():
    param.requires_grad = True

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=config['fine_tune_lr']
)

best_psnr = 0
for epoch in range(config['fine_tune_epochs']):
    model.train()
    epoch_losses = []

    with tqdm(train_loader, desc=f'Fine-tuning Epoch {epoch+1}/{config["fine_tune_epochs"]}') as pbar:
        for lr_imgs, hr_imgs in pbar:
            losses = trainer.fine_tuning_step((lr_imgs, hr_imgs), optimizer)
            epoch_losses.append(losses)
            pbar.set_postfix({k: f'{v:.4f}' for k, v in losses.items()})

    # Calculate average losses for epoch
    avg_losses = {k: sum(d[k] for d in epoch_losses) / len(epoch_losses)
                 for k in epoch_losses[0].keys()}

    # Validation
    if (epoch + 1) % config['val_frequency'] == 0:
        print(f"\nEpoch {epoch+1} average losses:")
        for k, v in avg_losses.items():
            print(f"{k}: {v:.4f}")

        print(f"\nValidating epoch {epoch+1}...")
        model.eval()
        psnr_values = []
        with torch.no_grad():
            for lr_imgs, hr_imgs in val_loader:
                lr_imgs, hr_imgs = lr_imgs.to(device), hr_imgs.to(device)
                sr_output, attention = model(lr_imgs)

                # Calculate PSNR
                mse = torch.mean((sr_output - hr_imgs) ** 2)
                psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))
                psnr_values.append(psnr.item())

        avg_psnr = sum(psnr_values) / len(psnr_values)
        print(f"Average PSNR: {avg_psnr:.2f}")

        # Save checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'psnr': avg_psnr,
        }, f'checkpoint_epoch_{epoch+1}.pt')

        # Save best model
        if avg_psnr > best_psnr:
            best_psnr = avg_psnr
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'psnr': best_psnr,
            }, 'best_model.pt')
            print(f"Saved new best model with PSNR {best_psnr:.2f}")

        # Visualize current results
        visualize_results(model, val_loader, device, f'results_epoch_{epoch+1}.png')

print("Training completed!")
print(f"Best PSNR achieved: {best_psnr:.2f}")

PSNR is still quite low, and we're seeing degradation in the RCAN output and a blank attention map. Have to diagnose the problem here.

In [ ]:
def check_attention_values(model, val_loader, device):
    model.eval()
    with torch.no_grad():
        lr_img, _ = next(iter(val_loader))
        lr_img = lr_img.to(device)
        _, attention = model(lr_img)
        print(f"Attention stats:")
        print(f"Min: {attention.min().item():.6f}")
        print(f"Max: {attention.max().item():.6f}")
        print(f"Mean: {attention.mean().item():.6f}")
        print(f"Std: {attention.std().item():.6f}")

# Load best model and check
checkpoint = torch.load('best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
check_attention_values(model, val_loader, device)

In [ ]:
def check_rcan_output(model, val_loader, device):
    model.eval()
    with torch.no_grad():
        lr_img, _ = next(iter(val_loader))
        lr_img = lr_img.to(device)
        output = model.rcan(lr_img)
        print(f"RCAN output stats:")
        print(f"Min: {output.min().item():.6f}")
        print(f"Max: {output.max().item():.6f}")
        print(f"Mean: {output.mean().item():.6f}")
        print(f"Std: {output.std().item():.6f}")

check_rcan_output(model, val_loader, device)

In [ ]:
def check_raw_rcan(device):
    print("Loading fresh RCAN model...")
    base_rcan = load_rcan_model()
    base_rcan = base_rcan.to(device)
    base_rcan.eval()

    with torch.no_grad():
        lr_img, _ = next(iter(val_loader))
        lr_img = lr_img.to(device)

        # Get output directly from RCAN
        output = base_rcan(lr_img)

        print("\nRaw RCAN output stats:")
        print(f"Min: {output.min().item():.6f}")
        print(f"Max: {output.max().item():.6f}")
        print(f"Mean: {output.mean().item():.6f}")
        print(f"Std: {output.std().item():.6f}")

        # Visualize without our wrapper
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.title('Raw RCAN Output')
        img = output.cpu().squeeze(0).permute(1, 2, 0).clamp(0, 1).numpy()
        plt.imshow(img)
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title('Green Channel')
        plt.imshow(img[:, :, 1], cmap='gray')
        plt.axis('off')
        plt.show()

check_raw_rcan(device)

Things look okay. Pushed some changes to ensure stability.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

class SelfSupervisedAttention(nn.Module):
    """Self-supervised auxiliary network for dynamic pixel importance prediction"""
    def __init__(self, in_channels=64):
        super().__init__()
        self.in_channels = in_channels

        # Spatial feature extraction
        self.conv1 = nn.Conv2d(in_channels, in_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv2 = nn.Conv2d(in_channels, in_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(in_channels)

        # Attention prediction
        self.conv3 = nn.Conv2d(in_channels, in_channels//2, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(in_channels//2)
        self.conv4 = nn.Conv2d(in_channels//2, 1, 1)

        # Channel attention
        self.spatial_pool = nn.AdaptiveAvgPool2d(1)
        self.channel_attention = nn.Sequential(
            nn.Linear(in_channels, in_channels//4),
            nn.ReLU(True),
            nn.Linear(in_channels//4, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Initial feature extraction
        feat = F.relu(self.bn1(self.conv1(x)))
        feat = F.relu(self.bn2(self.conv2(feat)))

        # Channel attention
        channel_att = self.spatial_pool(x).squeeze(-1).squeeze(-1)
        channel_att = self.channel_attention(channel_att)
        channel_att = channel_att.view(-1, self.in_channels, 1, 1)

        # Apply channel attention
        feat = feat * channel_att

        # Final attention prediction
        feat = F.relu(self.bn3(self.conv3(feat)))
        # Modified activation to ensure stronger attention values
        attention = torch.sigmoid(self.conv4(feat)) * 2  # Scale to [0,2] range

        return attention

class RCANFeatureExtractor(nn.Module):
    def __init__(self, rcan_model):
        super().__init__()
        self.rcan = rcan_model

    def extract_features(self, x):
        """Extract features before the final upsampling"""
        # Get initial features
        x = self.rcan.head(x)
        # Get body features
        body_feat = self.rcan.body(x)
        return x, body_feat

    def complete_sr(self, features):
        """Complete super-resolution with extracted features"""
        input_feat, body_feat = features
        # Residual connection
        x = body_feat + input_feat
        # Apply tail (upsampling)
        x = self.rcan.tail(x)
        # Ensure output is in valid range
        x = x.clamp(0, 1)
        return x

    def forward(self, x):
        features = self.extract_features(x)
        return self.complete_sr(features)

class AttentionAugmentedRCAN(nn.Module):
    def __init__(self, base_rcan, freeze_base=True):
        super().__init__()
        self.rcan = base_rcan
        self.attention_net = SelfSupervisedAttention(64)

        if freeze_base:
            for param in self.rcan.parameters():
                param.requires_grad = False

    def forward(self, x, mode='inference'):
        if mode == 'pre_training':
            feats = self.rcan.extract_features(x)
            attention = self.attention_net(feats[1])
            return attention

        # Normal forward pass with attention
        input_feat, body_feat = self.rcan.extract_features(x)
        attention = self.attention_net(body_feat)
        weighted_feat = body_feat * attention

        # Complete super-resolution
        sr_output = self.rcan.complete_sr((input_feat, weighted_feat))
        return sr_output, attention

class Trainer:
    def __init__(self, model, train_loader, val_loader, device):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.writer = SummaryWriter('runs/attention_rcan')

    def pre_training_step(self, batch, optimizer):
        lr_imgs, hr_imgs = [x.to(self.device) for x in batch]
        optimizer.zero_grad()

        # Get base RCAN output for difficulty estimation
        with torch.no_grad():
            sr_output = self.model.rcan(lr_imgs)
            difficulty = get_reconstruction_difficulty(sr_output, hr_imgs)

        # Train attention network
        attention = self.model(lr_imgs, mode='pre_training')

        # Upscale attention to match HR resolution
        attention_upscaled = F.interpolate(
            attention,
            size=difficulty.shape[-2:],
            mode='bilinear',
            align_corners=False
        )

        # Losses
        prediction_loss = F.mse_loss(attention_upscaled, difficulty)
        entropy_reg = entropy_loss(attention)
        consistency = spatial_consistency_loss(attention)

        total_loss = prediction_loss + 0.1 * entropy_reg + 0.1 * consistency

        total_loss.backward()
        optimizer.step()

        return {
            'prediction_loss': prediction_loss.item(),
            'entropy_loss': entropy_reg.item(),
            'consistency_loss': consistency.item(),
            'total_loss': total_loss.item()
        }

    def fine_tuning_step(self, batch, optimizer):
        lr_imgs, hr_imgs = [x.to(self.device) for x in batch]
        optimizer.zero_grad()

        # Forward pass with attention
        sr_output, attention = self.model(lr_imgs)

        # Calculate losses
        reconstruction_loss = F.l1_loss(sr_output, hr_imgs)
        entropy_reg = entropy_loss(attention)
        consistency = spatial_consistency_loss(attention)

        # Get current reconstruction difficulty
        difficulty = get_reconstruction_difficulty(sr_output, hr_imgs)

        # Upscale attention to match HR resolution
        attention_upscaled = F.interpolate(
            attention,
            size=difficulty.shape[-2:],
            mode='bilinear',
            align_corners=False
        )
        attention_guidance = F.mse_loss(attention_upscaled, difficulty.detach())

        total_loss = reconstruction_loss + 0.1 * entropy_reg + 0.1 * consistency + 0.1 * attention_guidance

        total_loss.backward()
        optimizer.step()

        return {
            'reconstruction_loss': reconstruction_loss.item(),
            'entropy_loss': entropy_reg.item(),
            'consistency_loss': consistency.item(),
            'attention_guidance': attention_guidance.item(),
            'total_loss': total_loss.item()
        }

def entropy_loss(attention_maps):
    """Entropy-based regularization for attention maps"""
    eps = 1e-8
    entropy = -(attention_maps * torch.log(attention_maps + eps))
    return entropy.mean()

def spatial_consistency_loss(attention_maps):
    """Spatial consistency loss for attention maps"""
    horizontal = F.l1_loss(attention_maps[..., :, 1:], attention_maps[..., :, :-1])
    vertical = F.l1_loss(attention_maps[..., 1:, :], attention_maps[..., :-1, :])
    return horizontal + vertical

def get_reconstruction_difficulty(sr_output, hr_target):
    """Calculate pixel-wise reconstruction difficulty"""
    with torch.no_grad():
        diff = torch.abs(sr_output - hr_target)
        # Normalize to [0, 1] range
        diff = (diff - diff.min()) / (diff.max() - diff.min() + 1e-8)
        return diff.mean(dim=1, keepdim=True)

def visualize_results(model, val_loader, device, save_path=None):
    """Visualize and compare results"""
    model.eval()
    with torch.no_grad():
        lr_img, hr_img = next(iter(val_loader))
        lr_img, hr_img = lr_img.to(device), hr_img.to(device)

        # Get outputs
        bicubic = F.interpolate(lr_img, scale_factor=4, mode='bicubic', align_corners=False)
        rcan_output = model.rcan(lr_img)
        sr_output, attention = model(lr_img)

        # Normalize attention for visualization
        attention = (attention - attention.min()) / (attention.max() - attention.min() + 1e-8)

        # Convert to images
        def tensor_to_image(x):
            x = x.clamp(0, 1)
            x = x.cpu().squeeze(0).permute(1, 2, 0).numpy()
            return x

        # Plot
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))

        axes[0, 0].imshow(tensor_to_image(lr_img))
        axes[0, 0].set_title('LR Input')
        axes[0, 1].imshow(tensor_to_image(bicubic))
        axes[0, 1].set_title('Bicubic')
        axes[0, 2].imshow(tensor_to_image(hr_img))
        axes[0, 2].set_title('HR Ground Truth')
        axes[1, 0].imshow(tensor_to_image(rcan_output))
        axes[1, 0].set_title('RCAN Output')
        axes[1, 1].imshow(tensor_to_image(sr_output))
        axes[1, 1].set_title('Our Model Output')
        axes[1, 2].imshow(tensor_to_image(attention.repeat(1, 3, 1, 1)))
        axes[1, 2].set_title('Attention Map')

        for ax in axes.flat:
            ax.axis('off')

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
        plt.show()

# Training configuration
config = {
    'pre_train_epochs': 50,
    'fine_tune_epochs': 100,
    'pre_train_lr': 1e-4,
    'fine_tune_lr': 1e-5,
    'val_frequency': 5
}

if __name__ == "__main__":
    # Setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load RCAN model
    print("Loading base RCAN model...")
    base_model = load_rcan_model()
    base_model = base_model.to(device)

    # Create augmented model
    print("Creating attention-augmented model...")
    model = AttentionAugmentedRCAN(base_model, freeze_base=True)
    model = model.to(device)

    # Setup data
    print("Setting up datasets...")
    train_loader, val_loader = setup_datasets(batch_size=16)

    # Create trainer
    trainer = Trainer(model, train_loader, val_loader, device)

Let's check that our baseline RCAN implementation works properly by comparing it to the official RCAN repo results.

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from math import log10
from PIL import Image
import torch.nn as nn
import cv2
from skimage.metrics import structural_similarity as ssim

def calc_psnr(sr, hr):
    """Calculate PSNR (Peak Signal-to-Noise Ratio)"""
    # Convert to numpy arrays in range [0, 255]
    sr = sr.mul(255).clamp(0, 255).round().cpu().numpy().transpose(1, 2, 0)
    hr = hr.mul(255).clamp(0, 255).round().cpu().numpy().transpose(1, 2, 0)

    # Calculate MSE
    mse = np.mean((sr - hr) ** 2)
    if mse == 0:
        return 100

    # Calculate PSNR
    psnr = 20 * log10(255.0 / np.sqrt(mse))
    return psnr

def calc_ssim(sr, hr):
    """Calculate SSIM (Structural Similarity Index)"""
    # Convert to numpy arrays in range [0, 255]
    sr = sr.mul(255).clamp(0, 255).round().cpu().numpy().transpose(1, 2, 0)
    hr = hr.mul(255).clamp(0, 255).round().cpu().numpy().transpose(1, 2, 0)

    return ssim(sr, hr, channel_axis=2, data_range=255)

def evaluate_rcan():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load RCAN model
    print("Loading RCAN model...")
    model = load_rcan_model()
    model = model.to(device)
    model.eval()

    # Setup validation data
    _, val_loader = setup_datasets(batch_size=1)  # Using Set14

    # Evaluation metrics
    psnr_values = []
    ssim_values = []

    print("\nEvaluating RCAN on Set14...")
    with torch.no_grad():
        for i, (lr_imgs, hr_imgs) in enumerate(val_loader):
            lr_imgs = lr_imgs.to(device)
            hr_imgs = hr_imgs.to(device)

            # Get RCAN output
            sr_output = model(lr_imgs)

            # Calculate metrics for each image
            for j in range(len(lr_imgs)):
                psnr = calc_psnr(sr_output[j], hr_imgs[j])
                ssim_val = calc_ssim(sr_output[j], hr_imgs[j])

                psnr_values.append(psnr)
                ssim_values.append(ssim_val)

                print(f"Image {i+1}: PSNR: {psnr:.2f} dB, SSIM: {ssim_val:.4f}")

    # Calculate average metrics
    avg_psnr = sum(psnr_values) / len(psnr_values)
    avg_ssim = sum(ssim_values) / len(ssim_values)

    print("\nOverall Results on Set14:")
    print(f"Average PSNR: {avg_psnr:.2f} dB")
    print(f"Average SSIM: {avg_ssim:.4f}")

    # Reference results from RCAN paper on Set14 (x4):
    print("\nReference Results from RCAN paper:")
    print("PSNR: 28.87 dB")
    print("SSIM: 0.7889")

if __name__ == "__main__":
    evaluate_rcan()

Performance is far below what it should be compared to our reference, so we'll need to debug.

In [ ]:
def check_image_ranges():
    _, val_loader = setup_datasets(batch_size=1)
    for lr_imgs, hr_imgs in val_loader:
        print(f"LR range: [{lr_imgs.min():.4f}, {lr_imgs.max():.4f}]")
        print(f"HR range: [{hr_imgs.min():.4f}, {hr_imgs.max():.4f}]")
        break

def print_rcan_args():
    # Print the args we're using for RCAN
    args = {
        'n_resgroups': 10,
        'n_resblocks': 20,
        'n_feats': 64,
        'scale': [4],
        'rgb_range': 255,  # This might be the issue
        'n_colors': 3,
        'res_scale': 1,
        'reduction': 16
    }
    print("Current RCAN args:")
    for k, v in args.items():
        print(f"{k}: {v}")

def check_model_output_range():
    device = torch.device('cuda')
    model = load_rcan_model().to(device)
    model.eval()

    _, val_loader = setup_datasets(batch_size=1)
    with torch.no_grad():
        lr_imgs, _ = next(iter(val_loader))
        lr_imgs = lr_imgs.to(device)
        output = model(lr_imgs)
        print(f"RCAN output range: [{output.min():.4f}, {output.max():.4f}]")

In [ ]:
check_image_ranges()
print_rcan_args()
check_model_output_range()

RGB range is set to 255 in the RCAN args, but we're feeding in 0,1 images.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
from PIL import Image
import numpy as np
from math import log10
import glob
import os
from google.colab import drive
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim

class SRDataset(Dataset):
    def __init__(self, root_dir, scale=4, patch_size=96, train=True):
        self.scale = scale
        self.patch_size = patch_size
        self.train = train

        # Mount Google Drive if not already mounted
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')

        # Find all images in the directory
        self.image_files = sorted(glob.glob(os.path.join(root_dir, '*.png')))
        if len(self.image_files) == 0:
            raise RuntimeError(f"No PNG images found in {root_dir}")

        print(f"Found {len(self.image_files)} images in {root_dir}")

        # Basic augmentations for training
        self.augment = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip()
        ]) if train else None

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load HR image
        img_path = self.image_files[idx]
        hr_image = Image.open(img_path).convert('RGB')

        # Handle training vs evaluation
        if self.train:
            # Random crop for training
            i, j, h, w = transforms.RandomCrop.get_params(
                hr_image, output_size=(self.patch_size, self.patch_size))
            hr_image = TF.crop(hr_image, i, j, h, w)

            # Apply augmentations
            if self.augment:
                hr_image = self.augment(hr_image)
        else:
            # For validation, ensure dimensions are divisible by scale
            w, h = hr_image.size
            w = w - w % self.scale
            h = h - h % self.scale
            hr_image = hr_image.crop((0, 0, w, h))

        # Convert to tensor and scale to [0, 255]
        hr_tensor = TF.to_tensor(hr_image) * 255.0

        # Create LR image using bicubic downsampling
        lr_tensor = TF.resize(
            hr_tensor,
            size=[s // self.scale for s in hr_tensor.shape[-2:]],
            interpolation=TF.InterpolationMode.BICUBIC
        )

        return lr_tensor, hr_tensor

def setup_datasets(batch_size=16):
    # Create datasets
    train_dataset = SRDataset(
        root_dir='DIV2K_train_HR',
        scale=4,
        patch_size=96,
        train=True
    )

    val_dataset = SRDataset(
        root_dir='/content/drive/MyDrive/E82/finalproject/Set14',
        scale=4,
        patch_size=96,
        train=False
    )

    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=1,
        pin_memory=True
    )

    return train_loader, val_loader

def calc_psnr(sr, hr):
    """Calculate PSNR (Peak Signal-to-Noise Ratio)"""
    # Convert to numpy arrays (values already in range [0, 255])
    sr = sr.clamp(0, 255).round().cpu().numpy().transpose(1, 2, 0)
    hr = hr.clamp(0, 255).round().cpu().numpy().transpose(1, 2, 0)

    # Calculate MSE
    mse = np.mean((sr - hr) ** 2)
    if mse == 0:
        return 100

    # Calculate PSNR
    psnr = 20 * log10(255.0 / np.sqrt(mse))
    return psnr

def calc_ssim(sr, hr):
    """Calculate SSIM (Structural Similarity Index)"""
    # Convert to numpy arrays (values already in range [0, 255])
    sr = sr.clamp(0, 255).round().cpu().numpy().transpose(1, 2, 0)
    hr = hr.clamp(0, 255).round().cpu().numpy().transpose(1, 2, 0)

    return ssim(sr, hr, channel_axis=2, data_range=255)

def evaluate_rcan():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Print RCAN configuration
    args = {
        'n_resgroups': 10,
        'n_resblocks': 20,
        'n_feats': 64,
        'scale': [4],
        'rgb_range': 255,
        'n_colors': 3,
        'res_scale': 1,
        'reduction': 16
    }
    print("\nRCAN Configuration:")
    for k, v in args.items():
        print(f"{k}: {v}")

    # Load RCAN model
    print("\nLoading RCAN model...")
    model = load_rcan_model()
    model = model.to(device)
    model.eval()

    # Setup validation data
    _, val_loader = setup_datasets(batch_size=1)

    # Check data ranges
    print("\nChecking data ranges...")
    lr_imgs, hr_imgs = next(iter(val_loader))
    print(f"LR range: [{lr_imgs.min():.4f}, {lr_imgs.max():.4f}]")
    print(f"HR range: [{hr_imgs.min():.4f}, {hr_imgs.max():.4f}]")

    # Evaluation metrics
    psnr_values = []
    ssim_values = []

    print("\nEvaluating RCAN on Set14...")
    with torch.no_grad():
        for i, (lr_imgs, hr_imgs) in enumerate(val_loader):
            lr_imgs = lr_imgs.to(device)
            hr_imgs = hr_imgs.to(device)

            # Get RCAN output
            sr_output = model(lr_imgs)

            # Check output range for first image
            if i == 0:
                print(f"SR output range: [{sr_output.min():.4f}, {sr_output.max():.4f}]")

            # Calculate metrics for each image
            for j in range(len(lr_imgs)):
                psnr = calc_psnr(sr_output[j], hr_imgs[j])
                ssim_val = calc_ssim(sr_output[j], hr_imgs[j])

                psnr_values.append(psnr)
                ssim_values.append(ssim_val)

                print(f"Image {i+1}: PSNR: {psnr:.2f} dB, SSIM: {ssim_val:.4f}")

    # Calculate average metrics
    avg_psnr = sum(psnr_values) / len(psnr_values)
    avg_ssim = sum(ssim_values) / len(ssim_values)

    print("\nOverall Results on Set14:")
    print(f"Average PSNR: {avg_psnr:.2f} dB")
    print(f"Average SSIM: {avg_ssim:.4f}")

    # Reference results from RCAN paper
    print("\nReference Results from RCAN paper:")
    print("PSNR: 28.87 dB")
    print("SSIM: 0.7889")

    # Calculate difference
    print("\nDifference from paper results:")
    print(f"PSNR diff: {28.87 - avg_psnr:.2f} dB")
    print(f"SSIM diff: {0.7889 - avg_ssim:.4f}")

if __name__ == "__main__":
    evaluate_rcan()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
from PIL import Image
import numpy as np
from math import log10
import glob
import os
from google.colab import drive
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim

class SRDataset(Dataset):
    def __init__(self, root_dir, scale=4, patch_size=96, train=True):
        self.scale = scale
        self.patch_size = patch_size
        self.train = train

        # Mount Google Drive if not already mounted
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')

        # Find all images in the directory
        self.image_files = sorted(glob.glob(os.path.join(root_dir, '*.png')))
        if len(self.image_files) == 0:
            raise RuntimeError(f"No PNG images found in {root_dir}")

        print(f"Found {len(self.image_files)} images in {root_dir}")

        # Basic augmentations for training
        self.augment = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip()
        ]) if train else None

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load HR image
        img_path = self.image_files[idx]
        hr_image = Image.open(img_path).convert('RGB')

        # Handle training vs evaluation
        if self.train:
            # Random crop for training
            i, j, h, w = transforms.RandomCrop.get_params(
                hr_image, output_size=(self.patch_size, self.patch_size))
            hr_image = TF.crop(hr_image, i, j, h, w)

            # Apply augmentations
            if self.augment:
                hr_image = self.augment(hr_image)
        else:
            # For validation, ensure dimensions are divisible by scale
            w, h = hr_image.size
            w = w - w % self.scale
            h = h - h % self.scale
            hr_image = hr_image.crop((0, 0, w, h))

        # Convert to tensor keeping in [0,255] range for metrics
        hr_tensor = TF.to_tensor(hr_image) * 255.0

        # Create LR image using bicubic downsampling
        lr_tensor = TF.resize(
            hr_tensor,
            size=[s // self.scale for s in hr_tensor.shape[-2:]],
            interpolation=TF.InterpolationMode.BICUBIC
        )

        return lr_tensor, hr_tensor

def setup_datasets(batch_size=16):
    """Setup training and validation dataloaders"""
    # Create datasets
    train_dataset = SRDataset(
        root_dir='DIV2K_train_HR',
        scale=4,
        patch_size=96,
        train=True
    )

    val_dataset = SRDataset(
        root_dir='/content/drive/MyDrive/E82/finalproject/Set14',
        scale=4,
        patch_size=96,
        train=False
    )

    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=1,
        pin_memory=True
    )

    return train_loader, val_loader

def calc_psnr(sr, hr):
    """Calculate PSNR (Peak Signal-to-Noise Ratio)"""
    # Values should already be in range [0, 255]
    sr = sr.clamp(0, 255).round().cpu().numpy().transpose(1, 2, 0)
    hr = hr.clamp(0, 255).round().cpu().numpy().transpose(1, 2, 0)

    # Calculate MSE
    mse = np.mean((sr - hr) ** 2)
    if mse == 0:
        return 100

    # Calculate PSNR
    psnr = 20 * log10(255.0 / np.sqrt(mse))
    return psnr

def calc_ssim(sr, hr):
    """Calculate SSIM (Structural Similarity Index)"""
    # Values should already be in range [0, 255]
    sr = sr.clamp(0, 255).round().cpu().numpy().transpose(1, 2, 0)
    hr = hr.clamp(0, 255).round().cpu().numpy().transpose(1, 2, 0)

    return ssim(sr, hr, channel_axis=2, data_range=255)

def evaluate_rcan():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Print RCAN configuration
    args = {
        'n_resgroups': 10,
        'n_resblocks': 20,
        'n_feats': 64,
        'scale': [4],
        'rgb_range': 255,
        'n_colors': 3,
        'res_scale': 1,
        'reduction': 16
    }
    print("\nRCAN Configuration:")
    for k, v in args.items():
        print(f"{k}: {v}")

    # Load RCAN model
    print("\nLoading RCAN model...")
    model = load_rcan_model()
    model = model.to(device)
    model.eval()

    # Setup validation data
    _, val_loader = setup_datasets(batch_size=1)

    # Evaluation metrics
    psnr_values = []
    ssim_values = []

    print("\nEvaluating RCAN on Set14...")
    with torch.no_grad():
        for i, (lr_imgs, hr_imgs) in enumerate(val_loader):
            # Move to device and normalize LR input to [0,1] range
            lr_imgs = lr_imgs.to(device) / 255.0
            hr_imgs = hr_imgs.to(device)  # Keep HR in [0,255] for metrics

            # Get RCAN output and scale back to [0,255] range
            sr_output = model(lr_imgs) * 255.0

            # Print ranges for first image
            if i == 0:
                print(f"\nFirst image ranges:")
                print(f"LR input range (after norm): [{lr_imgs.min():.4f}, {lr_imgs.max():.4f}]")
                print(f"SR output range (after scale): [{sr_output.min():.4f}, {sr_output.max():.4f}]")
                print(f"HR target range: [{hr_imgs.min():.4f}, {hr_imgs.max():.4f}]")

            # Calculate metrics
            psnr = calc_psnr(sr_output[0], hr_imgs[0])
            ssim_val = calc_ssim(sr_output[0], hr_imgs[0])

            psnr_values.append(psnr)
            ssim_values.append(ssim_val)
            print(f"Image {i+1}: PSNR: {psnr:.2f} dB, SSIM: {ssim_val:.4f}")

    # Calculate average metrics
    avg_psnr = sum(psnr_values) / len(psnr_values)
    avg_ssim = sum(ssim_values) / len(ssim_values)

    print("\nOverall Results on Set14:")
    print(f"Average PSNR: {avg_psnr:.2f} dB")
    print(f"Average SSIM: {avg_ssim:.4f}")

    # Reference results from RCAN paper
    print("\nReference Results from RCAN paper:")
    print("PSNR: 28.87 dB")
    print("SSIM: 0.7889")

    # Calculate difference
    print("\nDifference from paper results:")
    print(f"PSNR diff: {28.87 - avg_psnr:.2f} dB")
    print(f"SSIM diff: {0.7889 - avg_ssim:.4f}")

if __name__ == "__main__":
    evaluate_rcan()

In [ ]:
def evaluate_rcan_y_channel():
    """Evaluate RCAN using only Y channel (as done in paper)"""
    device = torch.device('cuda')
    model = load_rcan_model().to(device)
    model.eval()

    def rgb_to_y(img):
        """Convert RGB to Y channel (BT.709)"""
        if len(img.shape) == 4:
            img = img.squeeze(0)
        r, g, b = img[0], img[1], img[2]
        y = 0.2126 * r + 0.7152 * g + 0.0722 * b
        return y.unsqueeze(0).unsqueeze(0)

    _, val_loader = setup_datasets(batch_size=1)
    psnr_values = []

    print("\nEvaluating RCAN on Set14 (Y channel):")
    with torch.no_grad():
        for i, (lr_imgs, hr_imgs) in enumerate(val_loader):
            lr_imgs = lr_imgs.to(device) / 255.0
            hr_imgs = hr_imgs.to(device)  # Keep in [0,255]

            sr_output = model(lr_imgs) * 255.0

            # Convert to Y channel
            sr_y = rgb_to_y(sr_output[0])
            hr_y = rgb_to_y(hr_imgs[0])

            # Calculate PSNR on Y channel
            mse = torch.mean((sr_y - hr_y) ** 2)
            psnr = 20 * torch.log10(255.0 / torch.sqrt(mse))
            psnr_values.append(psnr.item())

            print(f"Image {i+1} Y-PSNR: {psnr:.2f} dB")

    avg_psnr = sum(psnr_values) / len(psnr_values)
    print(f"\nAverage Y-PSNR: {avg_psnr:.2f} dB")

evaluate_rcan_y_channel()

In [ ]:
def get_official_weights():
    print("Downloading official RCAN weights...")
    !wget https://www.dropbox.com/s/mjbcqkd4nwhr6nu/models_ECCV2018RCAN.zip
    !unzip models_ECCV2018RCAN.zip
    return './models_ECCV2018RCAN/RCAN_BIX4.pt'  # Path to official weights

# Use these weights instead
model_path = get_official_weights()

Fixed the RCAN implementation. Hopefully.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
from PIL import Image
import numpy as np
import math
from math import log10
import glob
import os
from google.colab import drive
from skimage.color import rgb2ycbcr

class MeanShift(nn.Conv2d):
    def __init__(self, rgb_range, rgb_mean, rgb_std, sign=-1):
        super(MeanShift, self).__init__(3, 3, kernel_size=1)
        std = torch.Tensor(rgb_std)
        self.weight.data = torch.eye(3).view(3, 3, 1, 1)
        self.weight.data.div_(std.view(3, 1, 1, 1))
        self.bias.data = sign * rgb_range * torch.Tensor(rgb_mean)
        self.bias.data.div_(std)
        self.requires_grad = False

class SRDataset(Dataset):
    def __init__(self, root_dir, scale=4, train=False):
        self.scale = scale

        # Find all images
        self.image_files = sorted(glob.glob(os.path.join(root_dir, '*.png')))
        if len(self.image_files) == 0:
            raise RuntimeError(f"No PNG images found in {root_dir}")

        print(f"Found {len(self.image_files)} images in {root_dir}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load HR image
        img_path = self.image_files[idx]
        hr_image = Image.open(img_path).convert('RGB')

        # Ensure dimensions are divisible by scale
        w, h = hr_image.size
        w = w - w % self.scale
        h = h - h % self.scale
        hr_image = hr_image.crop((0, 0, w, h))

        # Convert to tensor (keeping in [0, 255] initially)
        hr_tensor = TF.to_tensor(hr_image) * 255.0

        # Create LR using bicubic downsampling
        lr_tensor = TF.resize(
            hr_tensor,
            size=[s // self.scale for s in hr_tensor.shape[-2:]],
            interpolation=TF.InterpolationMode.BICUBIC
        )

        return lr_tensor, hr_tensor

def calc_psnr(sr, hr):
    """Calculate PSNR for Y channel"""
    def convert_rgb_to_y(img):
        if type(img) == np.ndarray:
            y = 16. + (64.738 * img[:, :, 0] + 129.057 * img[:, :, 1] + 25.064 * img[:, :, 2]) / 256.
            return y
        elif type(img) == torch.Tensor:
            if len(img.shape) == 4:
                img = img.squeeze(0)
            img = img.permute(1, 2, 0).cpu().numpy()
            y = 16. + (64.738 * img[:, :, 0] + 129.057 * img[:, :, 1] + 25.064 * img[:, :, 2]) / 256.
            return y
        else:
            raise Exception('Unknown Type', type(img))

    sr = convert_rgb_to_y(sr)
    hr = convert_rgb_to_y(hr)

    diff = (sr - hr) / 255.0
    mse = np.mean(diff ** 2)
    if mse == 0:
        return 100
    return -10 * log10(mse)

def evaluate_rcan():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Setup mean shift layers
    rgb_mean = (0.4488, 0.4371, 0.4040)
    rgb_std = (1.0, 1.0, 1.0)
    rgb_range = 255

    # Load RCAN model
    print("Loading RCAN model...")
    model = load_rcan_model()
    model = model.to(device)
    model.eval()

    # Setup validation data
    val_dataset = SRDataset('/content/drive/MyDrive/E82/finalproject/Set14', scale=4)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

    psnr_values = []
    print("\nEvaluating RCAN on Set14:")

    with torch.no_grad():
        for i, (lr_imgs, hr_imgs) in enumerate(val_loader):
            lr_imgs = lr_imgs.to(device)
            hr_imgs = hr_imgs.to(device)

            # Debug prints for first image
            if i == 0:
                print("\nTracking first image processing:")
                print(f"1. Initial LR range: [{lr_imgs.min():.4f}, {lr_imgs.max():.4f}]")

            # Normalize to [0, 1]
            lr_imgs = lr_imgs / 255.0

            if i == 0:
                print(f"2. After [0,1] norm: [{lr_imgs.min():.4f}, {lr_imgs.max():.4f}]")

            # Process through model (including its internal mean shifts)
            sr_output = model(lr_imgs)

            if i == 0:
                print(f"3. Model output: [{sr_output.min():.4f}, {sr_output.max():.4f}]")

            # Scale back to [0, 255]
            sr_output = sr_output * 255.0

            if i == 0:
                print(f"4. Final SR range: [{sr_output.min():.4f}, {sr_output.max():.4f}]")
                print(f"5. HR target range: [{hr_imgs.min():.4f}, {hr_imgs.max():.4f}]")

            # Calculate PSNR on Y channel
            psnr = calc_psnr(sr_output, hr_imgs)
            psnr_values.append(psnr)
            print(f"Image {i+1}: PSNR: {psnr:.2f} dB")

    avg_psnr = sum(psnr_values) / len(psnr_values)
    print(f"\nAverage PSNR: {avg_psnr:.2f} dB")
    print(f"Reference PSNR from paper: 28.87 dB")
    print(f"Difference: {28.87 - avg_psnr:.2f} dB")

if __name__ == "__main__":
    evaluate_rcan()

Getting significantly closer to the reference results.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
from PIL import Image
import numpy as np
from math import log10
import glob
import os
from google.colab import drive
import math

class SRDataset(Dataset):
    def __init__(self, root_dir, scale=4, train=False):
        self.scale = scale

        # Mount Google Drive if not already mounted
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')

        # Find all images
        self.image_files = sorted(glob.glob(os.path.join(root_dir, '*.png')))
        if len(self.image_files) == 0:
            raise RuntimeError(f"No PNG images found in {root_dir}")

        print(f"Found {len(self.image_files)} images in {root_dir}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load HR image
        img_path = self.image_files[idx]
        hr_image = Image.open(img_path).convert('RGB')

        # Ensure dimensions are divisible by scale
        w, h = hr_image.size
        w = w - w % self.scale
        h = h - h % self.scale
        hr_image = hr_image.crop((0, 0, w, h))

        # Convert to tensor (keeping in [0, 255] initially)
        hr_tensor = TF.to_tensor(hr_image) * 255.0

        # Create LR using bicubic downsampling
        lr_tensor = TF.resize(
            hr_tensor,
            size=[s // self.scale for s in hr_tensor.shape[-2:]],
            interpolation=TF.InterpolationMode.BICUBIC
        )

        return lr_tensor, hr_tensor

def setup_datasets(batch_size=16):
    """Setup training and validation dataloaders"""
    # Create datasets
    train_dataset = SRDataset(
        root_dir='DIV2K_train_HR',
        scale=4,
        train=True
    )

    val_dataset = SRDataset(
        root_dir='/content/drive/MyDrive/E82/finalproject/Set14',
        scale=4,
        train=False
    )

    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=1,
        pin_memory=True
    )

    return train_loader, val_loader

def evaluate_rcan():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    def quantize(img, rgb_range):
        pixel_range = 255 / rgb_range
        return img.mul(pixel_range).clamp(0, 255).round().div(pixel_range)

    def calc_psnr(sr, hr, scale=4, rgb_range=255):
        # Convert inputs from [0, 255] to [0, 1] by dividing by rgb_range
        diff = (sr - hr).data.div(rgb_range)
        shave = scale

        # Convert to Y
        if diff.size(1) > 1:
            convert = diff.new(1, 3, 1, 1)
            convert[0, 0, 0, 0] = 65.738
            convert[0, 1, 0, 0] = 129.057
            convert[0, 2, 0, 0] = 25.064
            diff.mul_(convert).div_(256)
            diff = diff.sum(dim=1, keepdim=True)

        # Shave borders
        valid = diff[:, :, shave:-shave, shave:-shave]
        mse = valid.pow(2).mean()
        return -10 * math.log10(mse)

    # Load RCAN model
    print("Loading RCAN model...")
    model = load_rcan_model()
    model = model.to(device)
    model.eval()

    # Setup validation data
    _, val_loader = setup_datasets(batch_size=1)

    psnr_values = []
    print("\nEvaluating RCAN on Set14:")
    with torch.no_grad():
        for i, (lr_imgs, hr_imgs) in enumerate(val_loader):
            lr_imgs = lr_imgs.to(device)
            hr_imgs = hr_imgs.to(device)

            # Scale to [0,1] range for model input
            lr_imgs = lr_imgs / 255.0

            # Get SR output
            sr_output = model(lr_imgs)

            # Scale back and quantize
            sr_output = sr_output * 255.0
            sr_output = quantize(sr_output, rgb_range=255)

            # Calculate PSNR using their method
            psnr = calc_psnr(sr_output, hr_imgs, scale=4, rgb_range=255)
            psnr_values.append(psnr)  # Removed .item() since psnr is already a float

            if i == 0:
                print(f"\nFirst image ranges:")
                print(f"LR input range (normalized): [{lr_imgs.min():.4f}, {lr_imgs.max():.4f}]")
                print(f"SR output range (after quantize): [{sr_output.min():.4f}, {sr_output.max():.4f}]")
                print(f"HR target range: [{hr_imgs.min():.4f}, {hr_imgs.max():.4f}]\n")

            print(f"Image {i+1}: PSNR: {psnr:.2f} dB")

    avg_psnr = sum(psnr_values) / len(psnr_values)
    print(f"\nAverage PSNR: {avg_psnr:.2f} dB")
    print(f"Reference PSNR from paper: 28.87 dB")
    print(f"Difference: {28.87 - avg_psnr:.2f} dB")

if __name__ == "__main__":
    evaluate_rcan()

In [ ]:
def evaluate_rcan():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    def quantize(img, rgb_range):
        pixel_range = 255 / rgb_range
        return img.mul(pixel_range).clamp(0, 255).round().div(pixel_range)

    def calc_psnr(sr, hr, scale=4, rgb_range=255):
        # Convert inputs from [0, 255] to [0, 1] by dividing by rgb_range
        diff = (sr - hr).data.div(rgb_range)
        shave = scale

        # Convert to Y
        if diff.size(1) > 1:
            convert = diff.new(1, 3, 1, 1)
            convert[0, 0, 0, 0] = 65.738
            convert[0, 1, 0, 0] = 129.057
            convert[0, 2, 0, 0] = 25.064
            diff.mul_(convert).div_(256)
            diff = diff.sum(dim=1, keepdim=True)

        # Shave borders
        valid = diff[:, :, shave:-shave, shave:-shave]
        mse = valid.pow(2).mean()
        return -10 * math.log10(mse)

    # Load RCAN model
    print("Loading RCAN model...")
    model = load_rcan_model()
    model = model.to(device)
    model.eval()

    # Setup validation data
    _, val_loader = setup_datasets(batch_size=1)

    print("\nStarting detailed evaluation:")
    with torch.no_grad():
        for i, (lr_imgs, hr_imgs) in enumerate(val_loader):
            lr_imgs = lr_imgs.to(device)
            hr_imgs = hr_imgs.to(device)

            print(f"\nStep 1 - Initial ranges:")
            print(f"LR range: [{lr_imgs.min():.4f}, {lr_imgs.max():.4f}]")
            print(f"HR range: [{hr_imgs.min():.4f}, {hr_imgs.max():.4f}]")

            # Scale to [0,1] range for model input
            lr_imgs = lr_imgs / 255.0
            print(f"\nStep 2 - After scaling LR to [0,1]:")
            print(f"LR range: [{lr_imgs.min():.4f}, {lr_imgs.max():.4f}]")

            # Get SR output
            sr_output = model(lr_imgs)
            print(f"\nStep 3 - Raw model output:")
            print(f"SR range: [{sr_output.min():.4f}, {sr_output.max():.4f}]")

            # Scale back and quantize
            sr_output = sr_output * 255.0
            print(f"\nStep 4 - After scaling back to [0,255]:")
            print(f"SR range: [{sr_output.min():.4f}, {sr_output.max():.4f}]")

            sr_output = quantize(sr_output, rgb_range=255)
            print(f"\nStep 5 - After quantization:")
            print(f"SR range: [{sr_output.min():.4f}, {sr_output.max():.4f}]")

            # Calculate PSNR
            print("\nStep 6 - Calculate PSNR:")
            print(f"Input shapes:")
            print(f"SR shape: {sr_output.shape}")
            print(f"HR shape: {hr_imgs.shape}")
            psnr = calc_psnr(sr_output, hr_imgs, scale=4, rgb_range=255)
            print(f"PSNR: {psnr:.2f} dB")

            # Check specific values
            print("\nStep 7 - Sample pixel values:")
            print("SR center pixel values:", sr_output[0, :, sr_output.shape[2]//2, sr_output.shape[3]//2])
            print("HR center pixel values:", hr_imgs[0, :, hr_imgs.shape[2]//2, hr_imgs.shape[3]//2])

            break  # Just examine first image

if __name__ == "__main__":
    evaluate_rcan()

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
from PIL import Image
import numpy as np
from math import log10
import glob
import os
from google.colab import drive
import math

class SRDataset(Dataset):
    def __init__(self, root_dir, scale=4, train=False):
        self.scale = scale

        # Mount Google Drive if not already mounted
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')

        # Find all images
        self.image_files = sorted(glob.glob(os.path.join(root_dir, '*.png')))
        if len(self.image_files) == 0:
            raise RuntimeError(f"No PNG images found in {root_dir}")

        print(f"Found {len(self.image_files)} images in {root_dir}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load HR image
        img_path = self.image_files[idx]
        hr_image = Image.open(img_path).convert('RGB')

        # Ensure dimensions are divisible by scale
        w, h = hr_image.size
        w = w - w % self.scale
        h = h - h % self.scale
        hr_image = hr_image.crop((0, 0, w, h))

        # Convert to tensor (keeping in [0, 255] initially)
        hr_tensor = TF.to_tensor(hr_image) * 255.0

        # Create LR using bicubic downsampling
        lr_tensor = TF.resize(
            hr_tensor,
            size=[s // self.scale for s in hr_tensor.shape[-2:]],
            interpolation=TF.InterpolationMode.BICUBIC
        )

        return lr_tensor, hr_tensor

def setup_datasets():
    """Setup validation dataloader for Set14"""
    val_dataset = SRDataset(
        root_dir='/content/drive/MyDrive/E82/finalproject/Set14',
        scale=4,
        train=False
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=1,
        pin_memory=True
    )

    return val_loader

def evaluate_rcan():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    def quantize(img, rgb_range):
        return img.clamp(0, 255).round()

    def calc_psnr(sr, hr, scale=4, rgb_range=255, benchmark=True):
        if hr.nelement() == 1:
            return 0

        diff = (sr - hr)
        shave = scale + 6 if not benchmark else scale
        if diff.size(1) > 1:
            gray_coeffs = [65.738, 129.057, 25.064]
            convert = diff.new_tensor(gray_coeffs).view(1, 3, 1, 1) / 256
            diff = diff.mul(convert).sum(dim=1, keepdim=True)

        valid = diff[:, :, shave:-shave, shave:-shave]
        mse = valid.pow(2).mean()

        return -10 * math.log10(mse/(rgb_range**2))

    # Load RCAN model
    print("Loading RCAN model...")
    model = load_rcan_model()
    model = model.to(device)
    model.eval()

    # Setup validation data
    val_loader = setup_datasets()

    print("\nStarting evaluation on Set14:")
    psnr_values = []

    with torch.no_grad():
        for i, (lr_imgs, hr_imgs) in enumerate(val_loader):
            if i == 0:
                print(f"\nStep 1 - Initial ranges:")
                print(f"LR range: [{lr_imgs.min():.4f}, {lr_imgs.max():.4f}]")
                print(f"HR range: [{hr_imgs.min():.4f}, {hr_imgs.max():.4f}]")

            lr_imgs = lr_imgs.to(device)
            hr_imgs = hr_imgs.to(device)

            # Scale to [0,1] range for model input
            lr_imgs = lr_imgs / 255.0

            if i == 0:
                print(f"\nStep 2 - After scaling LR to [0,1]:")
                print(f"LR range: [{lr_imgs.min():.4f}, {lr_imgs.max():.4f}]")

            # Get SR output
            sr_output = model(lr_imgs)

            if i == 0:
                print(f"\nStep 3 - Raw model output:")
                print(f"SR range: [{sr_output.min():.4f}, {sr_output.max():.4f}]")

            # Scale back to [0,255] and quantize
            sr_output = sr_output * 255.0
            sr_output = quantize(sr_output, rgb_range=255)

            if i == 0:
                print(f"\nStep 4 - After quantization:")
                print(f"SR range: [{sr_output.min():.4f}, {sr_output.max():.4f}]")
                print(f"Input shapes:")
                print(f"SR shape: {sr_output.shape}")
                print(f"HR shape: {hr_imgs.shape}")

            # Calculate PSNR
            psnr = calc_psnr(sr_output, hr_imgs, scale=4, rgb_range=255)
            psnr_values.append(psnr)

            if i == 0:
                print("\nStep 5 - Sample pixel values:")
                print("SR center pixel values:", sr_output[0, :, sr_output.shape[2]//2, sr_output.shape[3]//2])
                print("HR center pixel values:", hr_imgs[0, :, hr_imgs.shape[2]//2, hr_imgs.shape[3]//2])

            print(f"Image {i+1}: PSNR: {psnr:.2f} dB")

    avg_psnr = sum(psnr_values) / len(psnr_values)
    print(f"\nAverage PSNR: {avg_psnr:.2f} dB")
    print(f"Reference PSNR from paper: 28.87 dB")
    print(f"Difference: {28.87 - avg_psnr:.2f} dB")

if __name__ == "__main__":
    evaluate_rcan()

We've fixed the RCAN implementation as much as we reasonably can here. It uses the official implementation code wherever possible, so we can reasonably call it an optimized RCAN and move on to trying to beat it with our architecture.

In [ ]:
class SelfSupervisedAttention(nn.Module):
    """Self-supervised auxiliary network for dynamic pixel importance prediction"""
    def __init__(self, in_channels=64):
        super().__init__()
        self.in_channels = in_channels

        # Spatial feature extraction with BatchNorm for training stability
        self.conv1 = nn.Conv2d(in_channels, in_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv2 = nn.Conv2d(in_channels, in_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(in_channels)

        # Channel attention
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.channel_attention = nn.Sequential(
            nn.Linear(in_channels, in_channels//4),
            nn.ReLU(True),
            nn.Linear(in_channels//4, in_channels),
            nn.Sigmoid()
        )

        # Final attention prediction
        self.conv3 = nn.Conv2d(in_channels, in_channels//2, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(in_channels//2)
        self.conv4 = nn.Conv2d(in_channels//2, 1, 1)

        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        # Spatial features
        feat = F.relu(self.bn1(self.conv1(x)))
        feat = F.relu(self.bn2(self.conv2(feat)))

        # Channel attention
        channel_weights = self.avg_pool(feat).squeeze(-1).squeeze(-1)
        channel_weights = self.channel_attention(channel_weights)
        channel_weights = channel_weights.view(-1, self.in_channels, 1, 1)

        # Apply channel attention
        feat = feat * channel_weights

        # Generate attention map that enhances important features
        feat = F.relu(self.bn3(self.conv3(feat)))
        attention = torch.sigmoid(self.conv4(feat))

        # Scale attention to maintain feature magnitudes
        attention = attention + 1  # Range [1,2] to enhance features without destroying them

        return attention

class AttentionAugmentedRCAN(nn.Module):
    def __init__(self, base_rcan, freeze_base=True):
        super().__init__()
        self.rcan = base_rcan
        self.attention_net = SelfSupervisedAttention(64)  # RCAN uses 64 channels

        if freeze_base:
            for param in self.rcan.parameters():
                param.requires_grad = False

    def forward(self, x, mode='inference'):
        if mode == 'pre_training':
            # For attention network pre-training
            feats = self.rcan.extract_features(x)
            attention = self.attention_net(feats[1])  # Apply to body features
            return attention

        # Normal inference with attention
        input_feat, body_feat = self.rcan.extract_features(x)
        attention = self.attention_net(body_feat)
        weighted_feat = body_feat * attention

        # Complete super-resolution
        sr_output = self.rcan.complete_sr((input_feat, weighted_feat))

        return sr_output, attention

testing script to verify our attention mechanism is working correctly

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import figure

def test_attention_mechanism():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load base RCAN model
    print("Loading RCAN model...")
    base_model = load_rcan_model()
    base_model = base_model.to(device)

    # Create augmented model
    print("Creating attention-augmented model...")
    model = AttentionAugmentedRCAN(base_model, freeze_base=True)
    model = model.to(device)
    model.eval()

    # Create dummy input
    print("\nTesting with dummy input:")
    x = torch.randn(1, 3, 32, 32).to(device)
    x = (x - x.min()) / (x.max() - x.min())  # Normalize to [0,1]
    print(f"Input shape: {x.shape}")
    print(f"Input range: [{x.min():.4f}, {x.max():.4f}]")

    # Test pre-training mode
    print("\nTesting pre-training mode:")
    with torch.no_grad():
        attention = model(x, mode='pre_training')
        print(f"Attention shape: {attention.shape}")
        print(f"Attention range: [{attention.min():.4f}, {attention.max():.4f}]")

    # Test inference mode
    print("\nTesting inference mode:")
    with torch.no_grad():
        sr_output, attention = model(x)
        print(f"SR output shape: {sr_output.shape}")
        print(f"SR output range: [{sr_output.min():.4f}, {sr_output.max():.4f}]")
        print(f"Attention shape: {attention.shape}")
        print(f"Attention range: [{attention.min():.4f}, {attention.max():.4f}]")

    # Visualize attention map
    print("\nVisualizing attention map...")
    fig = plt.figure(figsize=(10, 5))

    # Heatmap
    plt.subplot(1, 2, 1)
    im = plt.imshow(attention[0, 0].cpu().numpy(), cmap='viridis')
    plt.colorbar(im)
    plt.title("Attention Map - Heatmap")

    # Histogram
    plt.subplot(1, 2, 2)
    plt.hist(attention.cpu().numpy().flatten(), bins=50)
    plt.title("Attention Map - Value Distribution")
    plt.tight_layout()
    plt.show()

    # Test gradients
    print("\nTesting gradient flow:")
    model.train()
    sr_output, attention = model(x)
    loss = sr_output.mean() + attention.mean()  # Dummy loss
    loss.backward()

    has_grad = lambda p: p.grad is not None and torch.abs(p.grad).sum().item() > 0

    print("Attention network gradients:")
    attention_grads = [has_grad(p) for p in model.attention_net.parameters()]
    print(f"Parameters with gradients: {sum(attention_grads)}/{len(attention_grads)}")

    print("\nRCAN gradients (should be zero if frozen):")
    rcan_grads = [has_grad(p) for p in model.rcan.parameters()]
    print(f"Parameters with gradients: {sum(rcan_grads)}/{len(rcan_grads)}")

    return model

if __name__ == "__main__":
    model = test_attention_mechanism()

Attention map shapes are correct (32x32 matching input features)

Attention values are in a reasonable range [1.26, 1.56], providing meaningful scaling

Gradient flow is working correctly:

All attention parameters (18/18) receive gradients

RCAN parameters (0/1630) are properly frozen


The attention distribution looks roughly Gaussian (from histogram)

Issues to address:

SR output has values outside [0,1] range: [-0.1375, 1.1809]

The attention map shows some vertical striping (visible in heatmap)

The attention range could be wider for stronger feature enhancement

In [ ]:
class SelfSupervisedAttention(nn.Module):
    def __init__(self, in_channels=64):
        super().__init__()
        self.in_channels = in_channels

        # Deeper feature extraction
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(True),
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(True)
        )

        # Channel attention with higher reduction ratio
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.channel_attention = nn.Sequential(
            nn.Linear(in_channels, in_channels//8),  # Increased reduction
            nn.ReLU(True),
            nn.Linear(in_channels//8, in_channels),
            nn.Sigmoid()
        )

        # Final attention prediction with smoother transition
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels, in_channels//2, 3, padding=1),
            nn.BatchNorm2d(in_channels//2),
            nn.ReLU(True),
            nn.Conv2d(in_channels//2, 1, 1),
        )

    def forward(self, x):
        # Feature extraction
        feat = self.conv1(x)

        # Channel attention
        channel_weights = self.avg_pool(feat).squeeze(-1).squeeze(-1)
        channel_weights = self.channel_attention(channel_weights)
        channel_weights = channel_weights.view(-1, self.in_channels, 1, 1)
        feat = feat * channel_weights

        # Generate attention map
        attention = self.conv2(feat)

        # Scale to [1.0, 2.0] with smoother activation
        attention = torch.tanh(attention) * 0.5 + 1.5

        return attention

class AttentionAugmentedRCAN(nn.Module):
    def __init__(self, base_rcan, freeze_base=True):
        super().__init__()
        self.rcan = base_rcan
        self.attention_net = SelfSupervisedAttention(64)

        if freeze_base:
            for param in self.rcan.parameters():
                param.requires_grad = False

    def forward(self, x, mode='inference'):
        if mode == 'pre_training':
            feats = self.rcan.extract_features(x)
            attention = self.attention_net(feats[1])
            return attention

        # Normal inference with attention
        input_feat, body_feat = self.rcan.extract_features(x)
        attention = self.attention_net(body_feat)
        weighted_feat = body_feat * attention

        # Complete super-resolution with clamping
        sr_output = self.rcan.complete_sr((input_feat, weighted_feat))
        sr_output = torch.clamp(sr_output, 0, 1)

        return sr_output, attention

In [ ]:
def test_updated_attention():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load base RCAN model
    print("Loading RCAN model...")
    base_model = load_rcan_model()
    base_model = base_model.to(device)

    # Create augmented model
    print("Creating attention-augmented model...")
    model = AttentionAugmentedRCAN(base_model, freeze_base=True)
    model = model.to(device)
    model.eval()

    # Test with real data from Set14
    print("\nTesting with real data:")
    val_loader = setup_datasets()
    lr_img, hr_img = next(iter(val_loader))
    lr_img, hr_img = lr_img.to(device), hr_img.to(device)

    # Scale input to [0,1]
    lr_img = lr_img / 255.0

    print(f"Input shape: {lr_img.shape}")
    print(f"Input range: [{lr_img.min():.4f}, {lr_img.max():.4f}]")

    # Test both modes
    print("\nTesting pre-training mode:")
    with torch.no_grad():
        attention = model(lr_img, mode='pre_training')
        print(f"Attention shape: {attention.shape}")
        print(f"Attention range: [{attention.min():.4f}, {attention.max():.4f}]")

    print("\nTesting inference mode:")
    with torch.no_grad():
        sr_output, attention = model(lr_img)
        print(f"SR output shape: {sr_output.shape}")
        print(f"SR output range: [{sr_output.min():.4f}, {sr_output.max():.4f}]")
        print(f"Attention shape: {attention.shape}")
        print(f"Attention range: [{attention.min():.4f}, {attention.max():.4f}]")

    # Visualize attention map
    print("\nVisualizing attention map...")
    fig = plt.figure(figsize=(15, 5))

    # Original LR image
    plt.subplot(1, 3, 1)
    lr_img_vis = lr_img[0].cpu().permute(1, 2, 0).numpy()
    plt.imshow(lr_img_vis)
    plt.title("LR Input")
    plt.axis('off')

    # Attention heatmap
    plt.subplot(1, 3, 2)
    im = plt.imshow(attention[0, 0].cpu().numpy(), cmap='viridis')
    plt.colorbar(im)
    plt.title("Attention Map")
    plt.axis('off')

    # Attention distribution
    plt.subplot(1, 3, 3)
    plt.hist(attention.cpu().numpy().flatten(), bins=50)
    plt.title("Attention Distribution")
    plt.tight_layout()
    plt.show()

    # Test gradients
    print("\nTesting gradient flow:")
    model.train()
    sr_output, attention = model(lr_img)

    # Use L1 loss as an example
    loss = F.l1_loss(sr_output, hr_img/255.0) + 0.1 * attention.mean()
    loss.backward()

    has_grad = lambda p: p.grad is not None and torch.abs(p.grad).sum().item() > 0

    print("Attention network gradients:")
    attention_grads = [has_grad(p) for p in model.attention_net.parameters()]
    print(f"Parameters with gradients: {sum(attention_grads)}/{len(attention_grads)}")

    print("\nRCAN gradients (should be zero if frozen):")
    rcan_grads = [has_grad(p) for p in model.rcan.parameters()]
    print(f"Parameters with gradients: {sum(rcan_grads)}/{len(rcan_grads)}")

    return model

if __name__ == "__main__":
    model = test_updated_attention()

Looking at the results, we have some interesting observations and potential improvements needed:
Good signs:

SR output is now properly bounded [0.0000, 1.0000]
The attention map shows clear structure matching the image content (you can see the baboon's features)
All attention parameters receive gradients (18/18)
RCAN remains properly frozen (0/1630)

Issues to address:

Attention range is very narrow [1.5407, 1.5523]

Our target was [1.0, 2.0] but we're only using about 1% of that range
This suggests the attention is not providing much feature enhancement


The attention distribution is highly peaked

Most values are clustered around 1.547
We want more variance to differentiate important features



Let's modify the attention network to address these:

In [ ]:
class SelfSupervisedAttention(nn.Module):
    def __init__(self, in_channels=64):
        super().__init__()
        self.in_channels = in_channels

        # Deeper feature extraction with more channels
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, in_channels*2, 3, padding=1),
            nn.BatchNorm2d(in_channels*2),
            nn.ReLU(True),
            nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(True)
        )

        # Channel attention with temperature scaling
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.channel_attention = nn.Sequential(
            nn.Linear(in_channels, in_channels//4),
            nn.ReLU(True),
            nn.Linear(in_channels//4, in_channels),
            nn.Sigmoid()
        )

        # Final attention prediction with temperature
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels, in_channels//2, 3, padding=1),
            nn.BatchNorm2d(in_channels//2),
            nn.ReLU(True),
            nn.Conv2d(in_channels//2, 1, 1),
        )

        self.temperature = nn.Parameter(torch.ones(1) * 0.1)  # Learnable temperature

    def forward(self, x):
        # Feature extraction
        feat = self.conv1(x)

        # Channel attention with temperature
        channel_weights = self.avg_pool(feat).squeeze(-1).squeeze(-1)
        channel_weights = self.channel_attention(channel_weights)
        channel_weights = channel_weights.view(-1, self.in_channels, 1, 1)
        feat = feat * channel_weights

        # Generate attention map with temperature scaling
        attention = self.conv2(feat)
        attention = torch.tanh(attention / self.temperature) * 0.5 + 1.5

        return attention

In [ ]:
def test_updated_attention():
   device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
   print(f"Using device: {device}")

   # Load base RCAN model
   print("Loading RCAN model...")
   base_model = load_rcan_model()
   base_model = base_model.to(device)

   # Create augmented model
   print("Creating attention-augmented model...")
   model = AttentionAugmentedRCAN(base_model, freeze_base=True)
   model = model.to(device)
   model.eval()

   # Test with real data from Set14
   print("\nTesting with real data:")
   val_loader = setup_datasets()

   # Test multiple images
   all_attention_values = []

   with torch.no_grad():
       for i, (lr_img, hr_img) in enumerate(val_loader):
           if i >= 3:  # Test first 3 images
               break

           lr_img = lr_img.to(device)
           lr_img = lr_img / 255.0  # Scale to [0,1]

           # Get model outputs
           sr_output, attention = model(lr_img)
           all_attention_values.append(attention.cpu().numpy())

           # Print stats for each image
           print(f"\nImage {i+1}:")
           print(f"Input shape: {lr_img.shape}")
           print(f"SR output shape: {sr_output.shape}")
           print(f"Attention range: [{attention.min():.4f}, {attention.max():.4f}]")
           print(f"Attention mean: {attention.mean():.4f}")
           print(f"Attention std: {attention.std():.4f}")

           # Visualize
           fig = plt.figure(figsize=(15, 5))

           # LR input
           plt.subplot(1, 3, 1)
           lr_img_vis = lr_img[0].cpu().permute(1, 2, 0).numpy()
           plt.imshow(lr_img_vis)
           plt.title(f"LR Input {i+1}")
           plt.axis('off')

           # Attention heatmap
           plt.subplot(1, 3, 2)
           attention_map = attention[0, 0].cpu().numpy()
           im = plt.imshow(attention_map, cmap='viridis')
           plt.colorbar(im)
           plt.title(f"Attention Map (μ={attention.mean():.3f}, σ={attention.std():.3f})")
           plt.axis('off')

           # Attention histogram
           plt.subplot(1, 3, 3)
           plt.hist(attention.cpu().numpy().flatten(), bins=50, density=True)
           plt.title("Attention Distribution")
           plt.axvline(attention.mean().item(), color='r', linestyle='--', label='Mean')
           plt.axvline(attention.mean().item() + attention.std().item(), color='g', linestyle='--', label='+1 std')
           plt.axvline(attention.mean().item() - attention.std().item(), color='g', linestyle='--', label='-1 std')
           plt.legend()

           plt.tight_layout()
           plt.show()

   # Print overall statistics
   all_attention_values = np.concatenate([a.flatten() for a in all_attention_values])
   print("\nOverall Attention Statistics:")
   print(f"Global range: [{np.min(all_attention_values):.4f}, {np.max(all_attention_values):.4f}]")
   print(f"Global mean: {np.mean(all_attention_values):.4f}")
   print(f"Global std: {np.std(all_attention_values):.4f}")

   # Test gradients
   print("\nTesting gradient flow...")
   model.train()
   lr_img, hr_img = next(iter(val_loader))
   lr_img = lr_img.to(device) / 255.0
   hr_img = hr_img.to(device) / 255.0

   sr_output, attention = model(lr_img)

   # Test with reconstruction loss and attention regularization
   recon_loss = F.l1_loss(sr_output, hr_img)
   attention_reg = 0.1 * (attention.mean() - 1.5).abs()  # Center around 1.5
   attention_std_reg = 0.1 * (0.1 - attention.std()).clamp(min=0)  # Encourage std > 0.1

   loss = recon_loss + attention_reg + attention_std_reg
   loss.backward()

   print("\nLoss components:")
   print(f"Reconstruction loss: {recon_loss.item():.4f}")
   print(f"Attention reg loss: {attention_reg.item():.4f}")
   print(f"Attention std reg loss: {attention_std_reg.item():.4f}")

   has_grad = lambda p: p.grad is not None and torch.abs(p.grad).sum().item() > 0

   print("\nGradient check:")
   print("Attention network gradients:")
   attention_grads = [has_grad(p) for p in model.attention_net.parameters()]
   print(f"Parameters with gradients: {sum(attention_grads)}/{len(attention_grads)}")

   # Check temperature gradient specifically
   temp_grad = model.attention_net.temperature.grad
   print(f"Temperature gradient: {temp_grad.item() if temp_grad is not None else 'None'}")

   return model

if __name__ == "__main__":
   model = test_updated_attention()

The attention mechanism is still not working as intended. There are several issues:

Attention Range:

Current: [1.023, 1.035] (variance of only ~0.012)
Target: [1.0, 2.0] (we want much more variance)
The attention maps are barely modulating the features


Standard Deviation:

Current: σ ≈ 0.001 (extremely small)
We need much larger variation to meaningfully enhance different features


Feature Detection:

The attention maps show some structure (visible in the heatmaps)
But the effect is too weak to meaningfully impact super-resolution



Let's modify the attention mechanism to be more aggressive:

In [ ]:
class SelfSupervisedAttention(nn.Module):
    def __init__(self, in_channels=64):
        super().__init__()
        self.in_channels = in_channels

        # Increase capacity for feature detection
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, in_channels*2, 3, padding=1),
            nn.BatchNorm2d(in_channels*2),
            nn.ReLU(True),
            nn.Conv2d(in_channels*2, in_channels*2, 3, padding=1),
            nn.BatchNorm2d(in_channels*2),
            nn.ReLU(True)
        )

        # Stronger channel attention
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)  # Add max pooling
        self.channel_attention = nn.Sequential(
            nn.Linear(in_channels*4, in_channels),  # Combine avg and max features
            nn.ReLU(True),
            nn.Linear(in_channels, in_channels*2),
            nn.Sigmoid()
        )

        # Final attention with stronger modulation
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(True),
            nn.Conv2d(in_channels, 1, 1),
        )

        # Learnable scaling parameters
        self.scale = nn.Parameter(torch.ones(1) * 0.5)  # Control output range
        self.temperature = nn.Parameter(torch.ones(1) * 0.1)  # Control sharpness

    def forward(self, x):
        # Feature extraction
        feat = self.conv1(x)

        # Enhanced channel attention
        avg_pool = self.avg_pool(feat).squeeze(-1).squeeze(-1)
        max_pool = self.max_pool(feat).squeeze(-1).squeeze(-1)
        channel_feats = torch.cat([avg_pool, max_pool], dim=1)
        channel_weights = self.channel_attention(channel_feats)
        channel_weights = channel_weights.view(-1, self.in_channels*2, 1, 1)

        feat = feat * channel_weights

        # Generate attention map with learnable scaling
        attention = self.conv2(feat)
        attention = torch.tanh(attention / self.temperature) * self.scale + 1.5

        return attention

class AttentionAugmentedRCAN(nn.Module):
    def __init__(self, base_rcan, freeze_base=True):
        super().__init__()
        self.rcan = base_rcan
        self.attention_net = SelfSupervisedAttention(64)

        if freeze_base:
            for param in self.rcan.parameters():
                param.requires_grad = False

        # Initialize with stronger attention
        self.attention_net.scale.data.fill_(1.0)
        self.attention_net.temperature.data.fill_(0.05)

    def forward(self, x, mode='inference'):
        if mode == 'pre_training':
            feats = self.rcan.extract_features(x)
            attention = self.attention_net(feats[1])
            return attention

        input_feat, body_feat = self.rcan.extract_features(x)
        attention = self.attention_net(body_feat)
        weighted_feat = body_feat * attention
        sr_output = self.rcan.complete_sr((input_feat, weighted_feat))
        return sr_output, attention

Testing again.

In [ ]:
def test_updated_attention():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load base RCAN model
    print("Loading RCAN model...")
    base_model = load_rcan_model()
    base_model = base_model.to(device)

    # Create augmented model
    print("Creating attention-augmented model...")
    model = AttentionAugmentedRCAN(base_model, freeze_base=True)
    model = model.to(device)
    model.eval()

    # Test with real data from Set14
    print("\nTesting with real data:")
    val_loader = setup_datasets()

    # Test multiple images
    all_attention_values = []

    with torch.no_grad():
        for i, (lr_img, hr_img) in enumerate(val_loader):
            if i >= 3:  # Test first 3 images
                break

            lr_img = lr_img.to(device)
            lr_img = lr_img / 255.0  # Scale to [0,1]

            # Get model outputs
            sr_output, attention = model(lr_img)
            all_attention_values.append(attention.cpu().numpy())

            # Print stats for each image
            print(f"\nImage {i+1}:")
            print(f"Input shape: {lr_img.shape}")
            print(f"SR output shape: {sr_output.shape}")
            print(f"Attention range: [{attention.min():.4f}, {attention.max():.4f}]")
            print(f"Attention mean: {attention.mean():.4f}")
            print(f"Attention std: {attention.std():.4f}")

            # Visualize
            fig = plt.figure(figsize=(15, 5))

            # LR input
            plt.subplot(1, 3, 1)
            lr_img_vis = lr_img[0].cpu().permute(1, 2, 0).numpy()
            plt.imshow(lr_img_vis)
            plt.title(f"LR Input {i+1}")
            plt.axis('off')

            # Attention heatmap
            plt.subplot(1, 3, 2)
            attention_map = attention[0, 0].cpu().numpy()
            im = plt.imshow(attention_map, cmap='viridis')
            plt.colorbar(im)
            plt.title(f"Attention Map (μ={attention.mean():.3f}, σ={attention.std():.3f})")
            plt.axis('off')

            # Attention histogram
            plt.subplot(1, 3, 3)
            plt.hist(attention.cpu().numpy().flatten(), bins=50, density=True)
            plt.title("Attention Distribution")
            plt.axvline(attention.mean().item(), color='r', linestyle='--', label='Mean')
            plt.axvline(attention.mean().item() + attention.std().item(), color='g', linestyle='--', label='+1 std')
            plt.axvline(attention.mean().item() - attention.std().item(), color='g', linestyle='--', label='-1 std')
            plt.legend()

            plt.tight_layout()
            plt.show()

    # Print overall statistics
    all_attention_values = np.concatenate([a.flatten() for a in all_attention_values])
    print("\nOverall Attention Statistics:")
    print(f"Global range: [{np.min(all_attention_values):.4f}, {np.max(all_attention_values):.4f}]")
    print(f"Global mean: {np.mean(all_attention_values):.4f}")
    print(f"Global std: {np.std(all_attention_values):.4f}")

    # Test gradients
    print("\nTesting gradient flow...")
    model.train()
    lr_img, hr_img = next(iter(val_loader))
    lr_img = lr_img.to(device) / 255.0
    hr_img = hr_img.to(device) / 255.0

    sr_output, attention = model(lr_img)

    # Test with reconstruction loss and attention regularization
    recon_loss = F.l1_loss(sr_output, hr_img)
    attention_reg = 0.1 * (attention.mean() - 1.5).abs()  # Center around 1.5
    attention_std_reg = 0.1 * (0.1 - attention.std()).clamp(min=0)  # Encourage std > 0.1

    loss = recon_loss + attention_reg + attention_std_reg
    loss.backward()

    print("\nLoss components:")
    print(f"Reconstruction loss: {recon_loss.item():.4f}")
    print(f"Attention reg loss: {attention_reg.item():.4f}")
    print(f"Attention std reg loss: {attention_std_reg.item():.4f}")

    has_grad = lambda p: p.grad is not None and torch.abs(p.grad).sum().item() > 0

    print("\nGradient check:")
    print("Attention network gradients:")
    attention_grads = [has_grad(p) for p in model.attention_net.parameters()]
    print(f"Parameters with gradients: {sum(attention_grads)}/{len(attention_grads)}")

    # Check scaling parameters specifically
    print("\nScaling parameters:")
    print(f"Scale value: {model.attention_net.scale.item():.4f}")
    print(f"Scale gradient: {model.attention_net.scale.grad.item() if model.attention_net.scale.grad is not None else 'None'}")
    print(f"Temperature value: {model.attention_net.temperature.item():.4f}")
    print(f"Temperature gradient: {model.attention_net.temperature.grad.item() if model.attention_net.temperature.grad is not None else 'None'}")

    return model

if __name__ == "__main__":
    model = test_updated_attention()

The attention values have shifted but we still have issues:

Attention Range:


Now [2.37, 2.44] - too high (outside our target [1.0, 2.0])
Still very narrow range (~0.07 difference)
Standard deviation remains tiny (σ ≈ 0.005)


Feature Detection:


Looking at the attention maps, we're getting similar feature detection as before
But now we're amplifying everything too much (mean ≈ 2.4)

Let's modify the attention mechanism to:

Better control the output range
Increase variance between important and less important regions

In [ ]:
class SelfSupervisedAttention(nn.Module):
    def __init__(self, in_channels=64):
        super().__init__()
        self.in_channels = in_channels

        # Feature extraction
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, in_channels*2, 3, padding=1),
            nn.BatchNorm2d(in_channels*2),
            nn.ReLU(True),
            nn.Conv2d(in_channels*2, in_channels*2, 3, padding=1),
            nn.BatchNorm2d(in_channels*2),
            nn.ReLU(True)
        )

        # Channel attention
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.channel_attention = nn.Sequential(
            nn.Linear(in_channels*4, in_channels),
            nn.ReLU(True),
            nn.Linear(in_channels, in_channels*2),
            nn.Sigmoid()
        )

        # Final attention
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(True),
            nn.Conv2d(in_channels, 1, 1),
        )

        # Initialize for moderate attention range
        self.base_attention = nn.Parameter(torch.ones(1) * 1.2)  # Base level
        self.attention_range = nn.Parameter(torch.ones(1) * 0.3)  # Max deviation
        self.temperature = nn.Parameter(torch.ones(1) * 0.1)

        self.scale = nn.Parameter(torch.tensor(1.0))  # Default scale factor

    def forward(self, x):
        # Feature extraction
        feat = self.conv1(x)

        # Enhanced channel attention
        avg_pool = self.avg_pool(feat).squeeze(-1).squeeze(-1)
        max_pool = self.max_pool(feat).squeeze(-1).squeeze(-1)
        channel_feats = torch.cat([avg_pool, max_pool], dim=1)
        channel_weights = self.channel_attention(channel_feats)
        channel_weights = channel_weights.view(-1, self.in_channels*2, 1, 1)

        feat = feat * channel_weights

        # Generate attention map
        attention_logits = self.conv2(feat)
        attention = torch.tanh(attention_logits / self.temperature)  # Range [-1, 1]

        # Scale to desired range around base_attention
        attention = self.base_attention + (attention * self.attention_range)

        return attention

In [ ]:
def has_grad(param):
    return param.grad is not None


def test_updated_attention():
   device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
   print(f"Using device: {device}")



   # Load base RCAN model
   print("Loading RCAN model...")
   base_model = load_rcan_model()
   base_model = base_model.to(device)

   # Create augmented model
   print("Creating attention-augmented model...")
   model = AttentionAugmentedRCAN(base_model, freeze_base=True)
   model = model.to(device)
   model.eval()

   # Test with real data from Set14
   print("\nTesting with real data:")
   val_loader = setup_datasets()

   # Test multiple images
   all_attention_values = []

   with torch.no_grad():
       for i, (lr_img, hr_img) in enumerate(val_loader):
           if i >= 3:  # Test first 3 images
               break

           lr_img = lr_img.to(device)
           lr_img = lr_img / 255.0  # Scale to [0,1]

           # Get model outputs
           sr_output, attention = model(lr_img)
           all_attention_values.append(attention.cpu().numpy())

           # Print stats for each image
           print(f"\nImage {i+1}:")
           print(f"Input shape: {lr_img.shape}")
           print(f"SR output shape: {sr_output.shape}")
           print(f"Attention range: [{attention.min():.4f}, {attention.max():.4f}]")
           print(f"Attention mean: {attention.mean():.4f}")
           print(f"Attention std: {attention.std():.4f}")

           # Visualize
           fig = plt.figure(figsize=(20, 5))

           # LR input
           plt.subplot(1, 4, 1)
           lr_img_vis = lr_img[0].cpu().permute(1, 2, 0).numpy()
           plt.imshow(lr_img_vis)
           plt.title(f"LR Input {i+1}")
           plt.axis('off')

           # SR output
           plt.subplot(1, 4, 2)
           sr_img_vis = sr_output[0].cpu().permute(1, 2, 0).numpy()
           plt.imshow(np.clip(sr_img_vis, 0, 1))
           plt.title("SR Output")
           plt.axis('off')

           # Attention heatmap
           plt.subplot(1, 4, 3)
           attention_map = attention[0, 0].cpu().numpy()
           im = plt.imshow(attention_map, cmap='viridis')
           plt.colorbar(im)
           plt.title(f"Attention Map (μ={attention.mean():.3f}, σ={attention.std():.3f})")
           plt.axis('off')

           # Attention histogram
           plt.subplot(1, 4, 4)
           plt.hist(attention.cpu().numpy().flatten(), bins=50, density=True)
           plt.title("Attention Distribution")
           plt.axvline(attention.mean().item(), color='r', linestyle='--', label='Mean')
           plt.axvline(attention.mean().item() + attention.std().item(), color='g', linestyle='--', label='+1 std')
           plt.axvline(attention.mean().item() - attention.std().item(), color='g', linestyle='--', label='-1 std')
           plt.axvline(model.attention_net.base_attention.item(), color='b', linestyle='--', label='Base Level')
           plt.legend()

           plt.tight_layout()
           plt.show()

   # Print overall statistics
   all_attention_values = np.concatenate([a.flatten() for a in all_attention_values])
   print("\nOverall Attention Statistics:")
   print(f"Global range: [{np.min(all_attention_values):.4f}, {np.max(all_attention_values):.4f}]")
   print(f"Global mean: {np.mean(all_attention_values):.4f}")
   print(f"Global std: {np.std(all_attention_values):.4f}")

   # Print attention parameters
   print("\nAttention Parameters:")
   print(f"Base attention: {model.attention_net.base_attention.item():.4f}")
   print(f"Attention range: {model.attention_net.attention_range.item():.4f}")
   print(f"Temperature: {model.attention_net.temperature.item():.4f}")

   # Test gradients
   print("\nTesting gradient flow...")
   model.train()
   lr_img, hr_img = next(iter(val_loader))
   lr_img = lr_img.to(device) / 255.0
   hr_img = hr_img.to(device) / 255.0

   sr_output, attention = model(lr_img)

   # Test with reconstruction loss and attention regularization
   recon_loss = F.l1_loss(sr_output, hr_img)
   attention_reg = 0.1 * (attention.mean() - model.attention_net.base_attention).abs()
   attention_std_reg = 0.1 * (0.1 - attention.std()).clamp(min=0)

   loss = recon_loss + attention_reg + attention_std_reg
   loss.backward()

   print("\nLoss components:")
   print(f"Reconstruction loss: {recon_loss.item():.4f}")
   print(f"Attention reg loss: {attention_reg.item():.4f}")
   print(f"Attention std reg loss: {attention_std_reg.item():.4f}")

   print("\nGradient check:")
   print("Attention network gradients:")
   attention_grads = [has_grad(p) for p in model.attention_net.parameters()]
   print(f"Parameters with gradients: {sum(attention_grads)}/{len(attention_grads)}")

   # Check parameter gradients
   print("\nParameter gradients:")
   print(f"Base attention grad: {model.attention_net.base_attention.grad.item():.4f}")
   print(f"Attention range grad: {model.attention_net.attention_range.grad.item():.4f}")
   print(f"Temperature grad: {model.attention_net.temperature.grad.item():.4f}")

   return model

if __name__ == "__main__":
   model = test_updated_attention()

Temperature:

Increased from 0.05 → 0.2 to smooth the attention map values, leading to less concentration around the base level.
Attention Regularization:

Reduced regularization weight (0.1 → 0.05) to allow the attention values to deviate more dynamically.
Attention Std Regularization:

Encourages a higher standard deviation (0.05 target) to ensure more variation in the attention map.

In [ ]:
class SelfSupervisedAttention(nn.Module):
    def __init__(self, in_channels=64):
        super().__init__()
        self.in_channels = in_channels

        # Feature extraction
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, in_channels*2, 3, padding=1),
            nn.BatchNorm2d(in_channels*2),
            nn.ReLU(True),
            nn.Conv2d(in_channels*2, in_channels*2, 3, padding=1),
            nn.BatchNorm2d(in_channels*2),
            nn.ReLU(True)
        )

        # Channel attention
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.channel_attention = nn.Sequential(
            nn.Linear(in_channels*4, in_channels),
            nn.ReLU(True),
            nn.Linear(in_channels, in_channels*2),
            nn.Sigmoid()
        )

        # Final attention
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(True),
            nn.Conv2d(in_channels, 1, 1),
        )

        # Initialize for moderate attention range
        self.base_attention = nn.Parameter(torch.ones(1) * 1.2)
        self.attention_range = nn.Parameter(torch.ones(1) * 0.3)
        self.temperature = nn.Parameter(torch.ones(1) * 0.2)  # Increased temperature
        self.scale = nn.Parameter(torch.tensor(1.0))  # Default scale

    def forward(self, x):
        # Feature extraction
        feat = self.conv1(x)

        # Enhanced channel attention
        avg_pool = self.avg_pool(feat).squeeze(-1).squeeze(-1)
        max_pool = self.max_pool(feat).squeeze(-1).squeeze(-1)
        channel_feats = torch.cat([avg_pool, max_pool], dim=1)
        channel_weights = self.channel_attention(channel_feats)
        channel_weights = channel_weights.view(-1, self.in_channels*2, 1, 1)

        feat = feat * channel_weights

        # Generate attention map
        attention_logits = self.conv2(feat)
        attention = torch.tanh(attention_logits / self.temperature)  # Range [-1, 1]

        # Scale to desired range around base_attention
        attention = self.base_attention + (attention * self.attention_range)

        return attention

In [ ]:
def has_grad(param):
    return param.grad is not None


def test_updated_attention():
   device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
   print(f"Using device: {device}")



   # Load base RCAN model
   print("Loading RCAN model...")
   base_model = load_rcan_model()
   base_model = base_model.to(device)

   # Create augmented model
   print("Creating attention-augmented model...")
   model = AttentionAugmentedRCAN(base_model, freeze_base=True)
   model = model.to(device)
   model.eval()

   # Test with real data from Set14
   print("\nTesting with real data:")
   val_loader = setup_datasets()

   # Test multiple images
   all_attention_values = []

   with torch.no_grad():
       for i, (lr_img, hr_img) in enumerate(val_loader):
           if i >= 3:  # Test first 3 images
               break

           lr_img = lr_img.to(device)
           lr_img = lr_img / 255.0  # Scale to [0,1]

           # Get model outputs
           sr_output, attention = model(lr_img)
           all_attention_values.append(attention.cpu().numpy())

           # Print stats for each image
           print(f"\nImage {i+1}:")
           print(f"Input shape: {lr_img.shape}")
           print(f"SR output shape: {sr_output.shape}")
           print(f"Attention range: [{attention.min():.4f}, {attention.max():.4f}]")
           print(f"Attention mean: {attention.mean():.4f}")
           print(f"Attention std: {attention.std():.4f}")

           # Visualize
           fig = plt.figure(figsize=(20, 5))

           # LR input
           plt.subplot(1, 4, 1)
           lr_img_vis = lr_img[0].cpu().permute(1, 2, 0).numpy()
           plt.imshow(lr_img_vis)
           plt.title(f"LR Input {i+1}")
           plt.axis('off')

           # SR output
           plt.subplot(1, 4, 2)
           sr_img_vis = sr_output[0].cpu().permute(1, 2, 0).numpy()
           plt.imshow(np.clip(sr_img_vis, 0, 1))
           plt.title("SR Output")
           plt.axis('off')

           # Attention heatmap
           plt.subplot(1, 4, 3)
           attention_map = attention[0, 0].cpu().numpy()
           im = plt.imshow(attention_map, cmap='viridis')
           plt.colorbar(im)
           plt.title(f"Attention Map (μ={attention.mean():.3f}, σ={attention.std():.3f})")
           plt.axis('off')

           # Attention histogram
           plt.subplot(1, 4, 4)
           plt.hist(attention.cpu().numpy().flatten(), bins=50, density=True)
           plt.title("Attention Distribution")
           plt.axvline(attention.mean().item(), color='r', linestyle='--', label='Mean')
           plt.axvline(attention.mean().item() + attention.std().item(), color='g', linestyle='--', label='+1 std')
           plt.axvline(attention.mean().item() - attention.std().item(), color='g', linestyle='--', label='-1 std')
           plt.axvline(model.attention_net.base_attention.item(), color='b', linestyle='--', label='Base Level')
           plt.legend()

           plt.tight_layout()
           plt.show()

   # Print overall statistics
   all_attention_values = np.concatenate([a.flatten() for a in all_attention_values])
   print("\nOverall Attention Statistics:")
   print(f"Global range: [{np.min(all_attention_values):.4f}, {np.max(all_attention_values):.4f}]")
   print(f"Global mean: {np.mean(all_attention_values):.4f}")
   print(f"Global std: {np.std(all_attention_values):.4f}")

   # Print attention parameters
   print("\nAttention Parameters:")
   print(f"Base attention: {model.attention_net.base_attention.item():.4f}")
   print(f"Attention range: {model.attention_net.attention_range.item():.4f}")
   print(f"Temperature: {model.attention_net.temperature.item():.4f}")

   # Test gradients
   print("\nTesting gradient flow...")
   model.train()
   lr_img, hr_img = next(iter(val_loader))
   lr_img = lr_img.to(device) / 255.0
   hr_img = hr_img.to(device) / 255.0

   sr_output, attention = model(lr_img)

   # Test with reconstruction loss and attention regularization
   recon_loss = F.l1_loss(sr_output, hr_img)
   attention_reg = 0.1 * (attention.mean() - model.attention_net.base_attention).abs()
   attention_std_reg = 0.1 * (0.1 - attention.std()).clamp(min=0)

   loss = recon_loss + attention_reg + attention_std_reg
   loss.backward()

   print("\nLoss components:")
   print(f"Reconstruction loss: {recon_loss.item():.4f}")
   print(f"Attention reg loss: {attention_reg.item():.4f}")
   print(f"Attention std reg loss: {attention_std_reg.item():.4f}")

   print("\nGradient check:")
   print("Attention network gradients:")
   attention_grads = [has_grad(p) for p in model.attention_net.parameters()]
   print(f"Parameters with gradients: {sum(attention_grads)}/{len(attention_grads)}")

   # Check parameter gradients
   print("\nParameter gradients:")
   print(f"Base attention grad: {model.attention_net.base_attention.grad.item():.4f}")
   print(f"Attention range grad: {model.attention_net.attention_range.grad.item():.4f}")
   print(f"Temperature grad: {model.attention_net.temperature.grad.item():.4f}")

   return model

if __name__ == "__main__":
   model = test_updated_attention()

In [ ]:
class SelfSupervisedAttention(nn.Module):
    def __init__(self, in_channels=64):
        super().__init__()
        self.in_channels = in_channels

        # Feature extraction
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, in_channels*2, 3, padding=1),
            nn.BatchNorm2d(in_channels*2),
            nn.LeakyReLU(0.2, True),  # LeakyReLU for better gradients
            nn.Conv2d(in_channels*2, in_channels*2, 3, padding=1),
            nn.BatchNorm2d(in_channels*2),
            nn.LeakyReLU(0.2, True)
        )

        # Channel attention with stronger modulation
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.channel_attention = nn.Sequential(
            nn.Linear(in_channels*4, in_channels),
            nn.LeakyReLU(0.2, True),
            nn.Linear(in_channels, in_channels*2),
            nn.Sigmoid()
        )

        # Final attention with direct range control
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
            nn.BatchNorm2d(in_channels),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(in_channels, 1, 1),
        )

        # More direct control parameters
        self.min_attention = nn.Parameter(torch.ones(1) * 0.8)  # Minimum attention
        self.max_attention = nn.Parameter(torch.ones(1) * 1.6)  # Maximum attention

    def forward(self, x):
        # Feature extraction
        feat = self.conv1(x)

        # Enhanced channel attention
        avg_pool = self.avg_pool(feat).squeeze(-1).squeeze(-1)
        max_pool = self.max_pool(feat).squeeze(-1).squeeze(-1)
        channel_feats = torch.cat([avg_pool, max_pool], dim=1)
        channel_weights = self.channel_attention(channel_feats)
        channel_weights = channel_weights.view(-1, self.in_channels*2, 1, 1)

        feat = feat * channel_weights

        # Generate attention map with direct range control
        attention_logits = self.conv2(feat)
        attention = torch.sigmoid(attention_logits)  # [0,1]

        # Scale directly to desired range
        attention = self.min_attention + (attention * (self.max_attention - self.min_attention))

        return attention

In [ ]:
def setup_datasets():
    """Setup validation dataloader for Set14 only"""
    val_dataset = SRDataset(
        root_dir='/content/drive/MyDrive/E82/finalproject/Set14',
        scale=4,
        train=False
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=1,
        pin_memory=True
    )

    return val_loader

class SRDataset(Dataset):
    def __init__(self, root_dir, scale=4, train=False):
        self.scale = scale

        # Mount Google Drive if not already mounted
        if not os.path.exists('/content/drive'):
            drive.mount('/content/drive')

        # Find all images
        self.image_files = sorted(glob.glob(os.path.join(root_dir, '*.png')))
        if len(self.image_files) == 0:
            raise RuntimeError(f"No PNG images found in {root_dir}")

        print(f"Found {len(self.image_files)} images in {root_dir}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load HR image
        img_path = self.image_files[idx]
        hr_image = Image.open(img_path).convert('RGB')

        # Ensure dimensions are divisible by scale
        w, h = hr_image.size
        w = w - w % self.scale
        h = h - h % self.scale
        hr_image = hr_image.crop((0, 0, w, h))

        # Convert to tensor (keeping in [0, 255] initially)
        hr_tensor = TF.to_tensor(hr_image) * 255.0

        # Create LR using bicubic downsampling
        lr_tensor = TF.resize(
            hr_tensor,
            size=[s // self.scale for s in hr_tensor.shape[-2:]],
            interpolation=TF.InterpolationMode.BICUBIC
        )

        return lr_tensor, hr_tensor

In [ ]:
def test_updated_attention():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load base RCAN model
    print("Loading RCAN model...")
    base_model = load_rcan_model()
    base_model = base_model.to(device)

    # Create augmented model
    print("Creating attention-augmented model...")
    model = AttentionAugmentedRCAN(base_model, freeze_base=True)
    model = model.to(device)
    model.eval()

    # Test with real data from Set14
    print("\nTesting with real data:")
    val_loader = setup_datasets()

    # Test multiple images
    all_attention_values = []

    with torch.no_grad():
        for i, (lr_img, hr_img) in enumerate(val_loader):
            if i >= 3:  # Test first 3 images
                break

            lr_img = lr_img.to(device)
            lr_img = lr_img / 255.0  # Scale to [0,1]

            # Get model outputs
            sr_output, attention = model(lr_img)
            all_attention_values.append(attention.cpu().numpy())

            # Print stats for each image
            print(f"\nImage {i+1}:")
            print(f"Input shape: {lr_img.shape}")
            print(f"SR output shape: {sr_output.shape}")
            print(f"Attention range: [{attention.min():.4f}, {attention.max():.4f}]")
            print(f"Attention mean: {attention.mean():.4f}")
            print(f"Attention std: {attention.std():.4f}")

            # Visualize
            fig = plt.figure(figsize=(20, 5))

            # LR input
            plt.subplot(1, 4, 1)
            lr_img_vis = lr_img[0].cpu().permute(1, 2, 0).numpy()
            plt.imshow(lr_img_vis)
            plt.title(f"LR Input {i+1}")
            plt.axis('off')

            # SR output
            plt.subplot(1, 4, 2)
            sr_img_vis = sr_output[0].cpu().permute(1, 2, 0).numpy()
            plt.imshow(np.clip(sr_img_vis, 0, 1))
            plt.title("SR Output")
            plt.axis('off')

            # Attention heatmap
            plt.subplot(1, 4, 3)
            attention_map = attention[0, 0].cpu().numpy()
            im = plt.imshow(attention_map, cmap='viridis')
            plt.colorbar(im)
            plt.title(f"Attention Map (μ={attention.mean():.3f}, σ={attention.std():.3f})")
            plt.axis('off')

            # Attention histogram
            plt.subplot(1, 4, 4)
            plt.hist(attention.cpu().numpy().flatten(), bins=50, density=True)
            plt.title("Attention Distribution")
            plt.axvline(attention.mean().item(), color='r', linestyle='--', label='Mean')
            plt.axvline(attention.mean().item() + attention.std().item(), color='g', linestyle='--', label='+1 std')
            plt.axvline(attention.mean().item() - attention.std().item(), color='g', linestyle='--', label='-1 std')
            plt.axvline(model.attention_net.min_attention.item(), color='b', linestyle='--', label='Min Level')
            plt.axvline(model.attention_net.max_attention.item(), color='b', linestyle='--', label='Max Level')
            plt.legend()

            plt.tight_layout()
            plt.show()

    # Print overall statistics
    all_attention_values = np.concatenate([a.flatten() for a in all_attention_values])
    print("\nOverall Attention Statistics:")
    print(f"Global range: [{np.min(all_attention_values):.4f}, {np.max(all_attention_values):.4f}]")
    print(f"Global mean: {np.mean(all_attention_values):.4f}")
    print(f"Global std: {np.std(all_attention_values):.4f}")

    # Print attention parameters
    print("\nAttention Parameters:")
    print(f"Min attention: {model.attention_net.min_attention.item():.4f}")
    print(f"Max attention: {model.attention_net.max_attention.item():.4f}")

    # Test gradients
    print("\nTesting gradient flow...")
    model.train()
    lr_img, hr_img = next(iter(val_loader))
    lr_img = lr_img.to(device) / 255.0
    hr_img = hr_img.to(device) / 255.0

    sr_output, attention = model(lr_img)

    # Test with reconstruction loss and attention regularization
    recon_loss = F.l1_loss(sr_output, hr_img)
    attention_std_reg = 0.1 * (0.1 - attention.std()).clamp(min=0)  # Encourage std > 0.1
    min_max_reg = 0.1 * F.l1_loss(attention, torch.ones_like(attention) * 1.2)  # Center around 1.2

    loss = recon_loss + attention_std_reg + min_max_reg
    loss.backward()

    print("\nLoss components:")
    print(f"Reconstruction loss: {recon_loss.item():.4f}")
    print(f"Attention std reg loss: {attention_std_reg.item():.4f}")
    print(f"Min-max reg loss: {min_max_reg.item():.4f}")

    print("\nGradient check:")
    print("Attention network gradients:")
    attention_grads = [has_grad(p) for p in model.attention_net.parameters()]
    print(f"Parameters with gradients: {sum(attention_grads)}/{len(attention_grads)}")

    # Check parameter gradients
    print("\nParameter gradients:")
    print(f"Min attention grad: {model.attention_net.min_attention.grad.item():.4f}")
    print(f"Max attention grad: {model.attention_net.max_attention.grad.item():.4f}")

    return model

if __name__ == "__main__":
    model = test_updated_attention()

The attention is still not working effectively. Even with our min/max range set to [0.8, 1.6], the actual values are tightly clustered around 1.19 with a tiny standard deviation (0.0004-0.0005). This suggests the sigmoid activation is saturating.
Let's modify the attention mechanism to be more direct:

In [ ]:
class SelfSupervisedAttention(nn.Module):
    def __init__(self, in_channels=64):
        super().__init__()
        self.in_channels = in_channels

        # Feature extraction with residual connections
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, in_channels*2, 3, padding=1),
            nn.BatchNorm2d(in_channels*2),
            nn.LeakyReLU(0.2, True)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels*2, in_channels*2, 3, padding=1),
            nn.BatchNorm2d(in_channels*2),
            nn.LeakyReLU(0.2, True)
        )

        # Channel attention
        self.channel_att = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels*2, in_channels//2, 1),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(in_channels//2, in_channels*2, 1),
            nn.Sigmoid()
        )

        # Direct attention prediction
        self.attention_conv = nn.Sequential(
            nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
            nn.BatchNorm2d(in_channels),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(in_channels, 1, 1)
        )

        # Explicit scale parameters
        self.attention_scale = nn.Parameter(torch.ones(1) * 0.4)  # Controls variation range
        self.attention_bias = nn.Parameter(torch.ones(1) * 1.2)   # Controls center point

    def forward(self, x):
        # Feature extraction with residual
        feat1 = self.conv1(x)
        feat2 = self.conv2(feat1)
        feat = feat1 + feat2

        # Apply channel attention
        channel_weights = self.channel_att(feat)
        feat = feat * channel_weights

        # Generate raw attention values
        attention_logits = self.attention_conv(feat)

        # Scale attention directly: bias ± scale * tanh(logits)
        attention = self.attention_bias + (self.attention_scale * torch.tanh(attention_logits))

        return attention

class AttentionAugmentedRCAN(nn.Module):
    def __init__(self, base_rcan, freeze_base=True):
        super().__init__()
        self.rcan = base_rcan
        self.attention_net = SelfSupervisedAttention(64)

        if freeze_base:
            for param in self.rcan.parameters():
                param.requires_grad = False

    def forward(self, x, mode='inference'):
        if mode == 'pre_training':
            feats = self.rcan.extract_features(x)
            attention = self.attention_net(feats[1])
            return attention

        input_feat, body_feat = self.rcan.extract_features(x)
        attention = self.attention_net(body_feat)
        weighted_feat = body_feat * attention
        sr_output = self.rcan.complete_sr((input_feat, weighted_feat))
        return sr_output, attention

Key changes:

Removed sigmoid bottleneck
Direct linear scaling using bias ± scale * tanh
Residual connections in feature extraction
Simplified channel attention
More explicit control over attention range

With bias=1.2 and scale=0.4, we should get:

Center point at 1.2
Range of approximately [0.8, 1.6] (bias ± scale)
More pronounced feature differentiation

In [ ]:
def test_updated_attention():
   device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
   print(f"Using device: {device}")

   # Load base RCAN model
   print("Loading RCAN model...")
   base_model = load_rcan_model()
   base_model = base_model.to(device)

   # Create augmented model
   print("Creating attention-augmented model...")
   model = AttentionAugmentedRCAN(base_model, freeze_base=True)
   model = model.to(device)
   model.eval()

   # Test with real data from Set14
   print("\nTesting with real data:")
   val_loader = setup_datasets()

   # Test multiple images
   all_attention_values = []

   with torch.no_grad():
       for i, (lr_img, hr_img) in enumerate(val_loader):
           if i >= 3:  # Test first 3 images
               break

           lr_img = lr_img.to(device)
           lr_img = lr_img / 255.0  # Scale to [0,1]

           # Get model outputs
           sr_output, attention = model(lr_img)
           all_attention_values.append(attention.cpu().numpy())

           # Print stats for each image
           print(f"\nImage {i+1}:")
           print(f"Input shape: {lr_img.shape}")
           print(f"SR output shape: {sr_output.shape}")
           print(f"Attention range: [{attention.min():.4f}, {attention.max():.4f}]")
           print(f"Attention mean: {attention.mean():.4f}")
           print(f"Attention std: {attention.std():.4f}")

           # Visualize
           fig = plt.figure(figsize=(20, 5))

           # LR input
           plt.subplot(1, 4, 1)
           lr_img_vis = lr_img[0].cpu().permute(1, 2, 0).numpy()
           plt.imshow(lr_img_vis)
           plt.title(f"LR Input {i+1}")
           plt.axis('off')

           # SR output
           plt.subplot(1, 4, 2)
           sr_img_vis = sr_output[0].cpu().permute(1, 2, 0).numpy()
           plt.imshow(np.clip(sr_img_vis, 0, 1))
           plt.title("SR Output")
           plt.axis('off')

           # Attention heatmap
           plt.subplot(1, 4, 3)
           attention_map = attention[0, 0].cpu().numpy()
           im = plt.imshow(attention_map, cmap='viridis')
           plt.colorbar(im)
           plt.title(f"Attention Map (μ={attention.mean():.3f}, σ={attention.std():.3f})")
           plt.axis('off')

           # Attention histogram
           plt.subplot(1, 4, 4)
           plt.hist(attention.cpu().numpy().flatten(), bins=50, density=True)
           plt.title("Attention Distribution")
           plt.axvline(attention.mean().item(), color='r', linestyle='--', label='Mean')
           plt.axvline(attention.mean().item() + attention.std().item(), color='g', linestyle='--', label='+1 std')
           plt.axvline(attention.mean().item() - attention.std().item(), color='g', linestyle='--', label='-1 std')
           plt.axvline(model.attention_net.attention_bias.item(), color='b', linestyle='--', label='Bias')
           plt.legend()

           plt.tight_layout()
           plt.show()

   # Print overall statistics
   all_attention_values = np.concatenate([a.flatten() for a in all_attention_values])
   print("\nOverall Attention Statistics:")
   print(f"Global range: [{np.min(all_attention_values):.4f}, {np.max(all_attention_values):.4f}]")
   print(f"Global mean: {np.mean(all_attention_values):.4f}")
   print(f"Global std: {np.std(all_attention_values):.4f}")

   # Print attention parameters
   print("\nAttention Parameters:")
   print(f"Bias: {model.attention_net.attention_bias.item():.4f}")
   print(f"Scale: {model.attention_net.attention_scale.item():.4f}")

   # Test gradients
   print("\nTesting gradient flow...")
   model.train()
   lr_img, hr_img = next(iter(val_loader))
   lr_img = lr_img.to(device) / 255.0
   hr_img = hr_img.to(device) / 255.0

   sr_output, attention = model(lr_img)

   # Test with reconstruction loss and attention regularization
   recon_loss = F.l1_loss(sr_output, hr_img)
   attention_std_reg = 0.1 * (0.1 - attention.std()).clamp(min=0)  # Encourage std > 0.1
   attention_reg = 0.1 * F.l1_loss(attention, torch.ones_like(attention) * 1.2)  # Center around target

   loss = recon_loss + attention_std_reg + attention_reg
   loss.backward()

   print("\nLoss components:")
   print(f"Reconstruction loss: {recon_loss.item():.4f}")
   print(f"Attention std reg loss: {attention_std_reg.item():.4f}")
   print(f"Attention reg loss: {attention_reg.item():.4f}")

   print("\nGradient check:")
   print("Attention network gradients:")
   attention_grads = [has_grad(p) for p in model.attention_net.parameters()]
   print(f"Parameters with gradients: {sum(attention_grads)}/{len(attention_grads)}")

   # Check parameter gradients
   print("\nParameter gradients:")
   print(f"Bias gradient: {model.attention_net.attention_bias.grad.item():.4f}")
   print(f"Scale gradient: {model.attention_net.attention_scale.grad.item():.4f}")

   return model

if __name__ == "__main__":
   model = test_updated_attention()

Not getting anywhere.  We will try an ensemble approach that combines multiple attention maps.

In [ ]:
def test_ensemble_attention():
   device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
   print(f"Using device: {device}")

   class SelfSupervisedAttention(nn.Module):
       def __init__(self, in_channels=64):
           super().__init__()
           self.in_channels = in_channels

           # Stronger feature extraction
           self.feat_conv = nn.Sequential(
               nn.Conv2d(in_channels, in_channels*2, 3, padding=1),
               nn.LeakyReLU(0.2, True),
               nn.Conv2d(in_channels*2, in_channels*2, 3, padding=1),
               nn.LeakyReLU(0.2, True)
           )

           # Multiple attention heads with different kernel sizes
           self.heads = nn.ModuleList([
               # Head 1: Local features (3x3)
               nn.Sequential(
                   nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels, 1, 3, padding=1)
               ),
               # Head 2: Medium features (5x5)
               nn.Sequential(
                   nn.Conv2d(in_channels*2, in_channels, 5, padding=2),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels, 1, 5, padding=2)
               ),
               # Head 3: Global features (7x7)
               nn.Sequential(
                   nn.Conv2d(in_channels*2, in_channels, 7, padding=3),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels, 1, 7, padding=3)
               )
           ])

           # Initialize head weights for different scales
           self.head_weights = nn.Parameter(torch.tensor([1.0, 0.7, 0.4]))

           # Range control
           self.min_attention = nn.Parameter(torch.tensor(0.8))
           self.max_attention = nn.Parameter(torch.tensor(1.6))

       def forward(self, x):
           # Extract features
           feat = self.feat_conv(x)

           # Get attention from each head
           attention_maps = [head(feat) for head in self.heads]

           # Combine with softmax weights
           weights = F.softmax(self.head_weights, dim=0)
           combined = sum(w * torch.tanh(m) for w, m in zip(weights, attention_maps))  # Use tanh for each head

           # Scale to desired range
           attention_range = self.max_attention - self.min_attention
           attention = self.min_attention + (torch.sigmoid(combined) * attention_range)

           return attention, attention_maps

   class AttentionAugmentedRCAN(nn.Module):
       def __init__(self, base_rcan, freeze_base=True):
           super().__init__()
           self.rcan = base_rcan
           self.attention_net = SelfSupervisedAttention(64)

           if freeze_base:
               for param in self.rcan.parameters():
                   param.requires_grad = False

       def forward(self, x, mode='inference'):
           if mode == 'pre_training':
               feats = self.rcan.extract_features(x)
               attention, _ = self.attention_net(feats[1])
               return attention

           input_feat, body_feat = self.rcan.extract_features(x)
           attention, attention_maps = self.attention_net(body_feat)
           weighted_feat = body_feat * attention
           sr_output = self.rcan.complete_sr((input_feat, weighted_feat))
           return sr_output, attention, attention_maps

   # Load RCAN model
   print("Loading RCAN model...")
   base_model = load_rcan_model()
   base_model = base_model.to(device)

   # Create augmented model
   print("Creating attention-augmented model...")
   model = AttentionAugmentedRCAN(base_model, freeze_base=True)
   model = model.to(device)
   model.eval()

   # Test with real data from Set14
   print("\nTesting with real data:")
   val_loader = setup_datasets()

   # Test multiple images
   all_attention_values = []

   with torch.no_grad():
       for i, (lr_img, hr_img) in enumerate(val_loader):
           if i >= 3:  # Test first 3 images
               break

           lr_img = lr_img.to(device)
           lr_img = lr_img / 255.0  # Scale to [0,1]

           # Get model outputs
           sr_output, attention, attention_maps = model(lr_img)
           all_attention_values.append(attention.cpu().numpy())

           # Print stats for each image
           print(f"\nImage {i+1}:")
           print(f"SR output shape: {sr_output.shape}")
           print(f"Attention range: [{attention.min():.4f}, {attention.max():.4f}]")
           print(f"Attention mean: {attention.mean():.4f}")
           print(f"Attention std: {attention.std():.4f}")

           # Print head weights and ranges
           weights = F.softmax(model.attention_net.head_weights, dim=0)
           print(f"Head weights: {weights.detach().cpu().numpy()}")
           for j, maps in enumerate(attention_maps):
               print(f"Head {j+1} range: [{maps.min():.4f}, {maps.max():.4f}]")

           # Visualize results
           fig = plt.figure(figsize=(20, 8))

           # LR input
           plt.subplot(2, 3, 1)
           lr_img_vis = lr_img[0].cpu().permute(1, 2, 0).numpy()
           plt.imshow(lr_img_vis)
           plt.title(f"LR Input {i+1}")
           plt.axis('off')

           # SR output
           plt.subplot(2, 3, 2)
           sr_img_vis = sr_output[0].cpu().permute(1, 2, 0).numpy()
           plt.imshow(np.clip(sr_img_vis, 0, 1))
           plt.title("SR Output")
           plt.axis('off')

           # Combined attention heatmap
           plt.subplot(2, 3, 3)
           attention_map = attention[0, 0].cpu().numpy()
           im = plt.imshow(attention_map, cmap='viridis')
           plt.colorbar(im)
           plt.title(f"Combined Attention (μ={attention.mean():.3f}, σ={attention.std():.3f})")
           plt.axis('off')

           # Individual head outputs
           for j, head_map in enumerate(attention_maps):
               plt.subplot(2, 3, 4+j)
               head_viz = head_map[0, 0].cpu().numpy()
               plt.imshow(head_viz, cmap='viridis')
               plt.colorbar()
               plt.title(f"Head {j+1} - {['3x3', '5x5', '7x7'][j]} (w: {weights[j]:.2f})")
               plt.axis('off')

           plt.tight_layout()
           plt.show()

   # Print overall statistics
   all_attention_values = np.concatenate([a.flatten() for a in all_attention_values])
   print("\nOverall Attention Statistics:")
   print(f"Global range: [{np.min(all_attention_values):.4f}, {np.max(all_attention_values):.4f}]")
   print(f"Global mean: {np.mean(all_attention_values):.4f}")
   print(f"Global std: {np.std(all_attention_values):.4f}")

   # Print attention parameters
   print("\nAttention Parameters:")
   print(f"Min attention: {model.attention_net.min_attention.item():.4f}")
   print(f"Max attention: {model.attention_net.max_attention.item():.4f}")

   return model

if __name__ == "__main__":
   model = test_ensemble_attention()

The results show that we're still not getting enough variation in the attention maps:

Attention Range Issues:

Range is narrow: [1.1985, 1.2043] (spread of only ~0.006)

Standard deviation is tiny: 0.0005

We're not using nearly the full range we set (0.8 to 1.6)


Head Behavior:

Head weights are working (different contributions: 44%, 32%, 24%)
Individual head ranges are very small (around ±0.04)
All heads seem to be producing similar patterns


Problems to Fix:

The sigmoid/tanh activations might be squashing our values too much
The range control isn't effective
The heads aren't learning sufficiently different features

Key changes:

Removed sigmoid/tanh activations
Direct scaling of head outputs
Different initial scales for each head (0.3, 0.2, 0.1)
Hard clamping to ensure reasonable range
Simpler combination strategy

In [ ]:
def test_direct_attention():
   device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
   print(f"Using device: {device}")

   class SelfSupervisedAttention(nn.Module):
       def __init__(self, in_channels=64):
           super().__init__()
           self.in_channels = in_channels

           # Feature extraction
           self.feat_conv = nn.Sequential(
               nn.Conv2d(in_channels, in_channels*2, 3, padding=1),
               nn.LeakyReLU(0.2, True),
               nn.Conv2d(in_channels*2, in_channels*2, 3, padding=1),
               nn.LeakyReLU(0.2, True)
           )

           # Three heads with different receptive fields
           self.heads = nn.ModuleList([
               # Local features
               nn.Sequential(
                   nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels, 1, 1)
               ),
               # Medium features
               nn.Sequential(
                   nn.Conv2d(in_channels*2, in_channels, 5, padding=2),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels, 1, 1)
               ),
               # Global features
               nn.Sequential(
                   nn.Conv2d(in_channels*2, in_channels, 7, padding=3),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels, 1, 1)
               )
           ])

           # Direct scaling parameters for each head
           self.head_scales = nn.Parameter(torch.tensor([0.3, 0.2, 0.1]))  # Different initial scales
           self.base_attention = nn.Parameter(torch.tensor(1.2))  # Base attention level

       def forward(self, x):
           # Extract features
           feat = self.feat_conv(x)

           # Get raw attention from each head
           attention_maps = [head(feat) for head in self.heads]

           # Scale and combine heads directly
           scaled_maps = []
           for map, scale in zip(attention_maps, self.head_scales):
               scaled_maps.append(map * scale)

           # Sum maps and add to base attention
           attention = self.base_attention + sum(scaled_maps)

           # Ensure reasonable range
           attention = attention.clamp(0.8, 1.6)

           return attention, attention_maps

   class AttentionAugmentedRCAN(nn.Module):
       def __init__(self, base_rcan, freeze_base=True):
           super().__init__()
           self.rcan = base_rcan
           self.attention_net = SelfSupervisedAttention(64)

           if freeze_base:
               for param in self.rcan.parameters():
                   param.requires_grad = False

       def forward(self, x, mode='inference'):
           if mode == 'pre_training':
               feats = self.rcan.extract_features(x)
               attention, _ = self.attention_net(feats[1])
               return attention

           input_feat, body_feat = self.rcan.extract_features(x)
           attention, attention_maps = self.attention_net(body_feat)
           weighted_feat = body_feat * attention
           sr_output = self.rcan.complete_sr((input_feat, weighted_feat))
           return sr_output, attention, attention_maps

   # Load RCAN model
   print("Loading RCAN model...")
   base_model = load_rcan_model()
   base_model = base_model.to(device)

   # Create augmented model
   print("Creating attention-augmented model...")
   model = AttentionAugmentedRCAN(base_model, freeze_base=True)
   model = model.to(device)
   model.eval()

   # Test with real data
   print("\nTesting with real data:")
   val_loader = setup_datasets()

   # Test multiple images
   all_attention_values = []

   with torch.no_grad():
       for i, (lr_img, hr_img) in enumerate(val_loader):
           if i >= 3:  # Test first 3 images
               break

           lr_img = lr_img.to(device)
           lr_img = lr_img / 255.0

           # Get model outputs
           sr_output, attention, attention_maps = model(lr_img)
           all_attention_values.append(attention.cpu().numpy())

           # Print stats
           print(f"\nImage {i+1}:")
           print(f"SR output shape: {sr_output.shape}")
           print(f"Attention range: [{attention.min():.4f}, {attention.max():.4f}]")
           print(f"Attention mean: {attention.mean():.4f}")
           print(f"Attention std: {attention.std():.4f}")

           # Print head scales and ranges
           print(f"Head scales: {model.attention_net.head_scales.detach().cpu().numpy()}")
           for j, maps in enumerate(attention_maps):
               print(f"Head {j+1} raw range: [{maps.min():.4f}, {maps.max():.4f}]")

           # Visualize
           fig = plt.figure(figsize=(20, 8))

           # Input and output
           plt.subplot(2, 3, 1)
           lr_img_vis = lr_img[0].cpu().permute(1, 2, 0).numpy()
           plt.imshow(lr_img_vis)
           plt.title(f"LR Input {i+1}")
           plt.axis('off')

           plt.subplot(2, 3, 2)
           sr_img_vis = sr_output[0].cpu().permute(1, 2, 0).numpy()
           plt.imshow(np.clip(sr_img_vis, 0, 1))
           plt.title("SR Output")
           plt.axis('off')

           # Attention maps
           plt.subplot(2, 3, 3)
           attention_map = attention[0, 0].cpu().numpy()
           im = plt.imshow(attention_map, cmap='viridis')
           plt.colorbar(im)
           plt.title(f"Combined Attention\nμ={attention.mean():.3f}, σ={attention.std():.3f}")
           plt.axis('off')

           # Individual head outputs
           for j, head_map in enumerate(attention_maps):
               plt.subplot(2, 3, 4+j)
               head_viz = head_map[0, 0].cpu().numpy()
               scale = model.attention_net.head_scales[j].item()
               plt.imshow(head_viz, cmap='viridis')
               plt.colorbar()
               plt.title(f"Head {j+1} (scale: {scale:.3f})")
               plt.axis('off')

           plt.tight_layout()
           plt.show()

   print("\nOverall Attention Statistics:")
   all_attention_values = np.concatenate([a.flatten() for a in all_attention_values])
   print(f"Global range: [{np.min(all_attention_values):.4f}, {np.max(all_attention_values):.4f}]")
   print(f"Global mean: {np.mean(all_attention_values):.4f}")
   print(f"Global std: {np.std(all_attention_values):.4f}")

   print("\nAttention Parameters:")
   print(f"Base attention: {model.attention_net.base_attention.item():.4f}")

   return model

if __name__ == "__main__":
   model = test_direct_attention()

Still having issues with very narrow attention range and std.


Key changes:

More channels in feature extraction (64→128→256)
Tanh activation for full [-1,1] range from heads
Doubled the scaling factors (0.3→0.6, etc.)
Wider clamping range (0.5 to 2.0)
Base attention at 1.0 for more room to move up/down

This should give us:

Much larger attention variations
Stronger feature enhancement
Clearer differentiation between regions

In [ ]:
def test_direct_attention():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    class SelfSupervisedAttention(nn.Module):
        def __init__(self, in_channels=64):
            super().__init__()
            self.in_channels = in_channels

            # Stronger feature extraction
            self.feat_conv = nn.Sequential(
                nn.Conv2d(in_channels, in_channels*2, 3, padding=1),
                nn.LeakyReLU(0.2, True),
                nn.Conv2d(in_channels*2, in_channels*4, 3, padding=1),  # More channels
                nn.LeakyReLU(0.2, True)
            )

            # Three heads with different receptive fields and stronger outputs
            self.heads = nn.ModuleList([
                # Local features (aggressive)
                nn.Sequential(
                    nn.Conv2d(in_channels*4, in_channels*2, 3, padding=1),
                    nn.LeakyReLU(0.2, True),
                    nn.Conv2d(in_channels*2, 1, 1),
                    nn.Tanh()  # Force full range
                ),
                # Medium features (balanced)
                nn.Sequential(
                    nn.Conv2d(in_channels*4, in_channels*2, 5, padding=2),
                    nn.LeakyReLU(0.2, True),
                    nn.Conv2d(in_channels*2, 1, 1),
                    nn.Tanh()
                ),
                # Global features (subtle)
                nn.Sequential(
                    nn.Conv2d(in_channels*4, in_channels*2, 7, padding=3),
                    nn.LeakyReLU(0.2, True),
                    nn.Conv2d(in_channels*2, 1, 1),
                    nn.Tanh()
                )
            ])

            # Much larger scales for more dramatic effect
            self.head_scales = nn.Parameter(torch.tensor([0.6, 0.4, 0.2]))  # Double the scales
            self.base_attention = nn.Parameter(torch.tensor(1.0))  # Start at 1.0

        def forward(self, x):
            # Extract features
            feat = self.feat_conv(x)

            # Get raw attention from each head (now in [-1,1] range due to tanh)
            attention_maps = [head(feat) for head in self.heads]

            # Scale and combine heads directly
            scaled_maps = []
            for map, scale in zip(attention_maps, self.head_scales):
                scaled_maps.append(map * scale)  # Will give ±0.6, ±0.4, ±0.2

            # Sum maps and add to base attention with wider range
            attention = self.base_attention + sum(scaled_maps)  # Range should be [0.0, 2.0]

            # Ensure reasonable range but allow more variation
            attention = attention.clamp(0.5, 2.0)

            return attention, attention_maps

    class AttentionAugmentedRCAN(nn.Module):
        def __init__(self, base_rcan, freeze_base=True):
            super().__init__()
            self.rcan = base_rcan
            self.attention_net = SelfSupervisedAttention(64)

            if freeze_base:
                for param in self.rcan.parameters():
                    param.requires_grad = False

        def forward(self, x, mode='inference'):
            if mode == 'pre_training':
                feats = self.rcan.extract_features(x)
                attention, _ = self.attention_net(feats[1])
                return attention

            input_feat, body_feat = self.rcan.extract_features(x)
            attention, attention_maps = self.attention_net(body_feat)
            weighted_feat = body_feat * attention
            sr_output = self.rcan.complete_sr((input_feat, weighted_feat))
            return sr_output, attention, attention_maps

    # Load RCAN model
    print("Loading RCAN model...")
    base_model = load_rcan_model()
    base_model = base_model.to(device)

    # Create augmented model
    print("Creating attention-augmented model...")
    model = AttentionAugmentedRCAN(base_model, freeze_base=True)
    model = model.to(device)
    model.eval()

    # Test with real data
    print("\nTesting with real data:")
    val_loader = setup_datasets()

    # Test multiple images
    all_attention_values = []

    with torch.no_grad():
        for i, (lr_img, hr_img) in enumerate(val_loader):
            if i >= 3:  # Test first 3 images
                break

            lr_img = lr_img.to(device)
            lr_img = lr_img / 255.0

            # Get model outputs
            sr_output, attention, attention_maps = model(lr_img)
            all_attention_values.append(attention.cpu().numpy())

            # Print stats
            print(f"\nImage {i+1}:")
            print(f"SR output shape: {sr_output.shape}")
            print(f"Attention range: [{attention.min():.4f}, {attention.max():.4f}]")
            print(f"Attention mean: {attention.mean():.4f}")
            print(f"Attention std: {attention.std():.4f}")

            # Print head scales and ranges
            print(f"Head scales: {model.attention_net.head_scales.detach().cpu().numpy()}")
            for j, maps in enumerate(attention_maps):
                print(f"Head {j+1} raw range: [{maps.min():.4f}, {maps.max():.4f}]")

            # Visualize
            fig = plt.figure(figsize=(20, 8))

            # Input and output
            plt.subplot(2, 3, 1)
            lr_img_vis = lr_img[0].cpu().permute(1, 2, 0).numpy()
            plt.imshow(lr_img_vis)
            plt.title(f"LR Input {i+1}")
            plt.axis('off')

            plt.subplot(2, 3, 2)
            sr_img_vis = sr_output[0].cpu().permute(1, 2, 0).numpy()
            plt.imshow(np.clip(sr_img_vis, 0, 1))
            plt.title("SR Output")
            plt.axis('off')

            # Attention maps
            plt.subplot(2, 3, 3)
            attention_map = attention[0, 0].cpu().numpy()
            im = plt.imshow(attention_map, cmap='viridis')
            plt.colorbar(im)
            plt.title(f"Combined Attention\nμ={attention.mean():.3f}, σ={attention.std():.3f}")
            plt.axis('off')

            # Individual head outputs
            for j, head_map in enumerate(attention_maps):
                plt.subplot(2, 3, 4+j)
                head_viz = head_map[0, 0].cpu().numpy()
                scale = model.attention_net.head_scales[j].item()
                plt.imshow(head_viz, cmap='viridis')
                plt.colorbar()
                plt.title(f"Head {j+1} (scale: {scale:.3f})")
                plt.axis('off')

            plt.tight_layout()
            plt.show()

    print("\nOverall Attention Statistics:")
    all_attention_values = np.concatenate([a.flatten() for a in all_attention_values])
    print(f"Global range: [{np.min(all_attention_values):.4f}, {np.max(all_attention_values):.4f}]")
    print(f"Global mean: {np.mean(all_attention_values):.4f}")
    print(f"Global std: {np.std(all_attention_values):.4f}")

    print("\nAttention Parameters:")
    print(f"Base attention: {model.attention_net.base_attention.item():.4f}")

    return model

if __name__ == "__main__":
    model = test_direct_attention()

Some minor improvement, but let's do a full test of the model now.

In [ ]:
def evaluate_for_paper():
   device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
   print(f"Using device: {device}")

   class SelfSupervisedAttention(nn.Module):
       def __init__(self, in_channels=64):
           super().__init__()
           self.in_channels = in_channels

           # Feature extraction
           self.feat_conv = nn.Sequential(
               nn.Conv2d(in_channels, in_channels*2, 3, padding=1),
               nn.LeakyReLU(0.2, True),
               nn.Conv2d(in_channels*2, in_channels*2, 3, padding=1),
               nn.LeakyReLU(0.2, True)
           )

           # Three heads with different receptive fields
           self.heads = nn.ModuleList([
               # Local features
               nn.Sequential(
                   nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels, 1, 1)
               ),
               # Medium features
               nn.Sequential(
                   nn.Conv2d(in_channels*2, in_channels, 5, padding=2),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels, 1, 1)
               ),
               # Global features
               nn.Sequential(
                   nn.Conv2d(in_channels*2, in_channels, 7, padding=3),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels, 1, 1)
               )
           ])

           # Direct scaling parameters for each head
           self.head_scales = nn.Parameter(torch.tensor([0.3, 0.2, 0.1]))  # Different initial scales
           self.base_attention = nn.Parameter(torch.tensor(1.2))  # Base attention level

       def forward(self, x):
           # Extract features
           feat = self.feat_conv(x)

           # Get raw attention from each head
           attention_maps = [head(feat) for head in self.heads]

           # Scale and combine heads directly
           scaled_maps = []
           for map, scale in zip(attention_maps, self.head_scales):
               scaled_maps.append(map * scale)

           # Sum maps and add to base attention
           attention = self.base_attention + sum(scaled_maps)

           # Ensure reasonable range
           attention = attention.clamp(0.8, 1.6)

           return attention, attention_maps

   class AttentionAugmentedRCAN(nn.Module):
       def __init__(self, base_rcan, freeze_base=True):
           super().__init__()
           self.rcan = base_rcan
           self.attention_net = SelfSupervisedAttention(64)

           if freeze_base:
               for param in self.rcan.parameters():
                   param.requires_grad = False

       def forward(self, x, mode='inference'):
           if mode == 'pre_training':
               feats = self.rcan.extract_features(x)
               attention, _ = self.attention_net(feats[1])
               return attention

           input_feat, body_feat = self.rcan.extract_features(x)
           attention, attention_maps = self.attention_net(body_feat)
           weighted_feat = body_feat * attention
           sr_output = self.rcan.complete_sr((input_feat, weighted_feat))
           return sr_output, attention

   # Load models
   print("Loading base RCAN...")
   base_model = load_rcan_model()
   base_model = base_model.to(device)
   base_model.eval()

   print("Loading attention-augmented RCAN...")
   att_model = AttentionAugmentedRCAN(base_model, freeze_base=True)
   att_model = att_model.to(device)
   att_model.eval()

   # Load validation data
   print("Loading Set14 dataset...")
   val_loader = setup_datasets()

   # Results storage
   results = []

   print("\nEvaluating models on Set14:")
   with torch.no_grad():
       for i, (lr_imgs, hr_imgs) in enumerate(val_loader):
           lr_imgs = lr_imgs.to(device)
           lr_input = lr_imgs / 255.0  # Scale to [0,1] for model input
           hr_imgs = hr_imgs.to(device)

           # Get outputs from both models
           base_sr = base_model(lr_input) * 255.0
           att_sr, attention = att_model(lr_input)
           att_sr = att_sr * 255.0

           # Calculate PSNR
           base_psnr = calc_psnr(base_sr, hr_imgs, scale=4, rgb_range=255)
           att_psnr = calc_psnr(att_sr, hr_imgs, scale=4, rgb_range=255)

           # Store results
           results.append({
               'image_idx': i+1,
               'base_psnr': base_psnr,
               'att_psnr': att_psnr,
               'improvement': att_psnr - base_psnr,
               'attention_stats': {
                   'min': attention.min().item(),
                   'max': attention.max().item(),
                   'mean': attention.mean().item(),
                   'std': attention.std().item()
               }
           })

           print(f"\nImage {i+1}:")
           print(f"Base RCAN PSNR: {base_psnr:.2f} dB")
           print(f"Attention RCAN PSNR: {att_psnr:.2f} dB")
           print(f"Improvement: {att_psnr - base_psnr:.2f} dB")

           # Save visual comparisons for first 3 images
           if i < 3:
               fig = plt.figure(figsize=(20, 5))

               # LR input
               plt.subplot(1, 4, 1)
               lr_img_vis = lr_imgs[0].cpu().permute(1, 2, 0).numpy().astype(np.uint8)
               plt.imshow(lr_img_vis)
               plt.title("LR Input")
               plt.axis('off')

               # Base RCAN output
               plt.subplot(1, 4, 2)
               base_sr_vis = base_sr[0].cpu().permute(1, 2, 0).numpy().astype(np.uint8)
               plt.imshow(base_sr_vis)
               plt.title(f"Base RCAN\nPSNR: {base_psnr:.2f} dB")
               plt.axis('off')

               # Attention RCAN output
               plt.subplot(1, 4, 3)
               att_sr_vis = att_sr[0].cpu().permute(1, 2, 0).numpy().astype(np.uint8)
               plt.imshow(att_sr_vis)
               plt.title(f"Attention RCAN\nPSNR: {att_psnr:.2f} dB")
               plt.axis('off')

               # Attention map
               plt.subplot(1, 4, 4)
               attention_map = attention[0, 0].cpu().numpy()
               im = plt.imshow(attention_map, cmap='viridis')
               plt.colorbar(im)
               plt.title(f"Attention Map\nRange: [{attention.min():.3f}, {attention.max():.3f}]")
               plt.axis('off')

               plt.tight_layout()
               plt.savefig(f'comparison_image_{i+1}.png', dpi=300, bbox_inches='tight')
               plt.show()

   # Calculate overall statistics
   base_psnrs = [r['base_psnr'] for r in results]
   att_psnrs = [r['att_psnr'] for r in results]
   improvements = [r['improvement'] for r in results]

   avg_base = np.mean(base_psnrs)
   avg_att = np.mean(att_psnrs)
   avg_imp = np.mean(improvements)

   print("\nOverall Results on Set14:")
   print(f"Average Base RCAN PSNR: {avg_base:.2f} dB")
   print(f"Average Attention RCAN PSNR: {avg_att:.2f} dB")
   print(f"Average Improvement: {avg_imp:.2f} dB")

   # Statistical significance test
   import scipy.stats as stats
   t_stat, p_value = stats.ttest_rel(att_psnrs, base_psnrs)
   print(f"\nStatistical Analysis:")
   print(f"T-statistic: {t_stat:.4f}")
   print(f"P-value: {p_value:.4f}")

   # Save detailed results to file
   with open('evaluation_results.txt', 'w') as f:
       f.write("Set14 Evaluation Results\n")
       f.write("=======================\n\n")
       f.write(f"Average Base RCAN PSNR: {avg_base:.2f} dB\n")
       f.write(f"Average Attention RCAN PSNR: {avg_att:.2f} dB\n")
       f.write(f"Average Improvement: {avg_imp:.2f} dB\n\n")
       f.write("Per-Image Results:\n")
       f.write("----------------\n")
       for r in results:
           f.write(f"\nImage {r['image_idx']}:\n")
           f.write(f"Base PSNR: {r['base_psnr']:.2f} dB\n")
           f.write(f"Attention PSNR: {r['att_psnr']:.2f} dB\n")
           f.write(f"Improvement: {r['improvement']:.2f} dB\n")
           f.write("Attention Stats:\n")
           for k, v in r['attention_stats'].items():
               f.write(f"  {k}: {v:.4f}\n")

   return results

if __name__ == "__main__":
   results = evaluate_for_paper()

Slightly worse results, but we haven't fully optimzied by any means.


The key changes to follow are:

4x wider feature channels
Deeper head networks

Tanh activation for full [-1,1] range

Much stronger scaling factors

Wider attention range (0.5 to 2.0)

In [ ]:
def evaluate_for_paper():
   device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
   print(f"Using device: {device}")

   class SelfSupervisedAttention(nn.Module):
       def __init__(self, in_channels=64):
           super().__init__()
           self.in_channels = in_channels

           # More powerful feature extraction
           self.feat_conv = nn.Sequential(
               nn.Conv2d(in_channels, in_channels*4, 3, padding=1),
               nn.LeakyReLU(0.2, True),
               nn.Conv2d(in_channels*4, in_channels*4, 3, padding=1),
               nn.LeakyReLU(0.2, True)
           )

           # Three heads with different receptive fields and stronger architecture
           self.heads = nn.ModuleList([
               # Local features (aggressive)
               nn.Sequential(
                   nn.Conv2d(in_channels*4, in_channels*2, 3, padding=1),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels, 1, 1),
                   nn.Tanh()
               ),
               # Medium features
               nn.Sequential(
                   nn.Conv2d(in_channels*4, in_channels*2, 5, padding=2),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels, 1, 1),
                   nn.Tanh()
               ),
               # Global features
               nn.Sequential(
                   nn.Conv2d(in_channels*4, in_channels*2, 7, padding=3),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
                   nn.LeakyReLU(0.2, True),
                   nn.Conv2d(in_channels, 1, 1),
                   nn.Tanh()
               )
           ])

           # Much stronger scaling factors
           self.head_scales = nn.Parameter(torch.tensor([1.0, 0.7, 0.4]))
           self.base_attention = nn.Parameter(torch.tensor(1.0))

       def forward(self, x):
           # Extract features
           feat = self.feat_conv(x)

           # Get raw attention from each head (now in [-1,1] range due to tanh)
           attention_maps = [head(feat) for head in self.heads]

           # Scale and combine heads directly
           scaled_maps = []
           for map, scale in zip(attention_maps, self.head_scales):
               scaled_maps.append(map * scale)

           # Sum maps and add to base attention
           attention = self.base_attention + sum(scaled_maps)

           # Allow wider range
           attention = attention.clamp(0.5, 2.0)

           return attention, attention_maps

   class AttentionAugmentedRCAN(nn.Module):
       def __init__(self, base_rcan, freeze_base=True):
           super().__init__()
           self.rcan = base_rcan
           self.attention_net = SelfSupervisedAttention(64)

           if freeze_base:
               for param in self.rcan.parameters():
                   param.requires_grad = False

       def forward(self, x, mode='inference'):
           if mode == 'pre_training':
               feats = self.rcan.extract_features(x)
               attention, _ = self.attention_net(feats[1])
               return attention

           input_feat, body_feat = self.rcan.extract_features(x)
           attention, attention_maps = self.attention_net(body_feat)
           weighted_feat = body_feat * attention
           sr_output = self.rcan.complete_sr((input_feat, weighted_feat))
           return sr_output, attention

   # Load models
   print("Loading base RCAN...")
   base_model = load_rcan_model()
   base_model = base_model.to(device)
   base_model.eval()

   print("Loading attention-augmented RCAN...")
   att_model = AttentionAugmentedRCAN(base_model, freeze_base=True)
   att_model = att_model.to(device)
   att_model.eval()

   # Load validation data
   print("Loading Set14 dataset...")
   val_loader = setup_datasets()

   # Results storage
   results = []

   print("\nEvaluating models on Set14:")
   with torch.no_grad():
       for i, (lr_imgs, hr_imgs) in enumerate(val_loader):
           lr_imgs = lr_imgs.to(device)
           lr_input = lr_imgs / 255.0  # Scale to [0,1] for model input
           hr_imgs = hr_imgs.to(device)

           # Get outputs from both models
           base_sr = base_model(lr_input) * 255.0
           att_sr, attention = att_model(lr_input)
           att_sr = att_sr * 255.0

           # Calculate PSNR
           base_psnr = calc_psnr(base_sr, hr_imgs, scale=4, rgb_range=255)
           att_psnr = calc_psnr(att_sr, hr_imgs, scale=4, rgb_range=255)

           # Store results
           results.append({
               'image_idx': i+1,
               'base_psnr': base_psnr,
               'att_psnr': att_psnr,
               'improvement': att_psnr - base_psnr,
               'attention_stats': {
                   'min': attention.min().item(),
                   'max': attention.max().item(),
                   'mean': attention.mean().item(),
                   'std': attention.std().item()
               }
           })

           print(f"\nImage {i+1}:")
           print(f"Base RCAN PSNR: {base_psnr:.2f} dB")
           print(f"Attention RCAN PSNR: {att_psnr:.2f} dB")
           print(f"Improvement: {att_psnr - base_psnr:.2f} dB")

           # Save visual comparisons for first 3 images
           if i < 3:
               fig = plt.figure(figsize=(20, 5))

               # LR input
               plt.subplot(1, 4, 1)
               lr_img_vis = lr_imgs[0].cpu().permute(1, 2, 0).numpy().astype(np.uint8)
               plt.imshow(lr_img_vis)
               plt.title("LR Input")
               plt.axis('off')

               # Base RCAN output
               plt.subplot(1, 4, 2)
               base_sr_vis = base_sr[0].cpu().permute(1, 2, 0).numpy().astype(np.uint8)
               plt.imshow(base_sr_vis)
               plt.title(f"Base RCAN\nPSNR: {base_psnr:.2f} dB")
               plt.axis('off')

               # Attention RCAN output
               plt.subplot(1, 4, 3)
               att_sr_vis = att_sr[0].cpu().permute(1, 2, 0).numpy().astype(np.uint8)
               plt.imshow(att_sr_vis)
               plt.title(f"Attention RCAN\nPSNR: {att_psnr:.2f} dB")
               plt.axis('off')

               # Attention map
               plt.subplot(1, 4, 4)
               attention_map = attention[0, 0].cpu().numpy()
               im = plt.imshow(attention_map, cmap='viridis')
               plt.colorbar(im)
               plt.title(f"Attention Map\nRange: [{attention.min():.3f}, {attention.max():.3f}]")
               plt.axis('off')

               plt.tight_layout()
               plt.savefig(f'comparison_image_{i+1}.png', dpi=300, bbox_inches='tight')
               plt.show()

   # Calculate overall statistics
   base_psnrs = [r['base_psnr'] for r in results]
   att_psnrs = [r['att_psnr'] for r in results]
   improvements = [r['improvement'] for r in results]

   avg_base = np.mean(base_psnrs)
   avg_att = np.mean(att_psnrs)
   avg_imp = np.mean(improvements)

   print("\nOverall Results on Set14:")
   print(f"Average Base RCAN PSNR: {avg_base:.2f} dB")
   print(f"Average Attention RCAN PSNR: {avg_att:.2f} dB")
   print(f"Average Improvement: {avg_imp:.2f} dB")

   # Statistical significance test
   import scipy.stats as stats
   t_stat, p_value = stats.ttest_rel(att_psnrs, base_psnrs)
   print(f"\nStatistical Analysis:")
   print(f"T-statistic: {t_stat:.4f}")
   print(f"P-value: {p_value:.4f}")

   # Save detailed results to file
   with open('evaluation_results.txt', 'w') as f:
       f.write("Set14 Evaluation Results\n")
       f.write("=======================\n\n")
       f.write(f"Average Base RCAN PSNR: {avg_base:.2f} dB\n")
       f.write(f"Average Attention RCAN PSNR: {avg_att:.2f} dB\n")
       f.write(f"Average Improvement: {avg_imp:.2f} dB\n\n")
       f.write("Per-Image Results:\n")
       f.write("----------------\n")
       for r in results:
           f.write(f"\nImage {r['image_idx']}:\n")
           f.write(f"Base PSNR: {r['base_psnr']:.2f} dB\n")
           f.write(f"Attention PSNR: {r['att_psnr']:.2f} dB\n")
           f.write(f"Improvement: {r['improvement']:.2f} dB\n")
           f.write("Attention Stats:\n")
           for k, v in r['attention_stats'].items():
               f.write(f"  {k}: {v:.4f}\n")

   return results

if __name__ == "__main__":
   results = evaluate_for_paper()

We do see an improvement over our baseline RCAN model now.

In [ ]:
def evaluate_for_paper():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    class SelfSupervisedAttention(nn.Module):
        def __init__(self, in_channels=64):
            super().__init__()
            self.in_channels = in_channels

            # More powerful feature extraction
            self.feat_conv = nn.Sequential(
                nn.Conv2d(in_channels, in_channels*4, 3, padding=1),
                nn.LeakyReLU(0.2, True),
                nn.Conv2d(in_channels*4, in_channels*4, 3, padding=1),
                nn.LeakyReLU(0.2, True)
            )

            # Three heads with different receptive fields and stronger architecture
            self.heads = nn.ModuleList([
                # Local features (aggressive)
                nn.Sequential(
                    nn.Conv2d(in_channels*4, in_channels*2, 3, padding=1),
                    nn.LeakyReLU(0.2, True),
                    nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
                    nn.LeakyReLU(0.2, True),
                    nn.Conv2d(in_channels, 1, 1),
                    nn.Tanh()
                ),
                # Medium features
                nn.Sequential(
                    nn.Conv2d(in_channels*4, in_channels*2, 5, padding=2),
                    nn.LeakyReLU(0.2, True),
                    nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
                    nn.LeakyReLU(0.2, True),
                    nn.Conv2d(in_channels, 1, 1),
                    nn.Tanh()
                ),
                # Global features
                nn.Sequential(
                    nn.Conv2d(in_channels*4, in_channels*2, 7, padding=3),
                    nn.LeakyReLU(0.2, True),
                    nn.Conv2d(in_channels*2, in_channels, 3, padding=1),
                    nn.LeakyReLU(0.2, True),
                    nn.Conv2d(in_channels, 1, 1),
                    nn.Tanh()
                )
            ])

            # Much stronger scaling factors
            self.head_scales = nn.Parameter(torch.tensor([1.0, 0.7, 0.4]))
            self.base_attention = nn.Parameter(torch.tensor(1.0))

        def forward(self, x):
            # Extract features
            feat = self.feat_conv(x)

            # Get raw attention from each head (now in [-1,1] range due to tanh)
            attention_maps = [head(feat) for head in self.heads]

            # Scale and combine heads directly
            scaled_maps = []
            for map, scale in zip(attention_maps, self.head_scales):
                scaled_maps.append(map * scale)

            # Sum maps and add to base attention
            attention = self.base_attention + sum(scaled_maps)

            # Allow wider range
            attention = attention.clamp(0.5, 2.0)

            return attention, attention_maps

    class AttentionAugmentedRCAN(nn.Module):
        def __init__(self, base_rcan, freeze_base=True):
            super().__init__()
            self.rcan = base_rcan
            self.attention_net = SelfSupervisedAttention(64)

            if freeze_base:
                for param in self.rcan.parameters():
                    param.requires_grad = False

        def forward(self, x, mode='inference'):
            if mode == 'pre_training':
                feats = self.rcan.extract_features(x)
                attention, _ = self.attention_net(feats[1])
                return attention

            input_feat, body_feat = self.rcan.extract_features(x)
            attention, attention_maps = self.attention_net(body_feat)
            weighted_feat = body_feat * attention
            sr_output = self.rcan.complete_sr((input_feat, weighted_feat))
            return sr_output, attention

    # Load models
    print("Loading base RCAN...")
    base_model = load_rcan_model()
    base_model = base_model.to(device)
    base_model.eval()

    print("Loading attention-augmented RCAN...")
    att_model = AttentionAugmentedRCAN(base_model, freeze_base=True)
    att_model = att_model.to(device)
    att_model.eval()

    # Load validation data
    print("Loading Set14 dataset...")
    val_loader = setup_datasets()

    # Results storage
    results = []

    print("\nEvaluating models on Set14:")
    with torch.no_grad():
        for i, (lr_imgs, hr_imgs) in enumerate(val_loader):
            lr_imgs = lr_imgs.to(device)
            lr_input = lr_imgs / 255.0  # Scale to [0,1] for model input
            hr_imgs = hr_imgs.to(device)

            # Get outputs from both models
            base_sr = base_model(lr_input) * 255.0
            att_sr, attention = att_model(lr_input)
            att_sr = att_sr * 255.0

            # Calculate PSNR
            base_psnr = calc_psnr(base_sr, hr_imgs, scale=4, rgb_range=255)
            att_psnr = calc_psnr(att_sr, hr_imgs, scale=4, rgb_range=255)

            # Store results
            results.append({
                'image_idx': i+1,
                'base_psnr': base_psnr,
                'att_psnr': att_psnr,
                'improvement': att_psnr - base_psnr,
                'attention_stats': {
                    'min': attention.min().item(),
                    'max': attention.max().item(),
                    'mean': attention.mean().item(),
                    'std': attention.std().item()
                }
            })

            print(f"\nImage {i+1}:")
            print(f"Base RCAN PSNR: {base_psnr:.2f} dB")
            print(f"Attention RCAN PSNR: {att_psnr:.2f} dB")
            print(f"Improvement: {att_psnr - base_psnr:.2f} dB")

            # Save visual comparisons for all images
            fig = plt.figure(figsize=(20, 5))

            # LR input
            plt.subplot(1, 4, 1)
            lr_img_vis = lr_imgs[0].cpu().permute(1, 2, 0).numpy().astype(np.uint8)
            plt.imshow(lr_img_vis)
            plt.title("LR Input")
            plt.axis('off')

            # Base RCAN output
            plt.subplot(1, 4, 2)
            base_sr_vis = base_sr[0].cpu().permute(1, 2, 0).numpy().astype(np.uint8)
            plt.imshow(base_sr_vis)
            plt.title(f"Base RCAN\nPSNR: {base_psnr:.2f} dB")
            plt.axis('off')

            # Attention RCAN output
            plt.subplot(1, 4, 3)
            att_sr_vis = att_sr[0].cpu().permute(1, 2, 0).numpy().astype(np.uint8)
            plt.imshow(att_sr_vis)
            plt.title(f"Attention RCAN\nPSNR: {att_psnr:.2f} dB")
            plt.axis('off')

            # Attention map
            plt.subplot(1, 4, 4)
            attention_map = attention[0, 0].cpu().numpy()
            im = plt.imshow(attention_map, cmap='viridis')
            plt.colorbar(im)
            plt.title(f"Attention Map\nRange: [{attention.min():.3f}, {attention.max():.3f}]")
            plt.axis('off')

            plt.tight_layout()
            plt.savefig(f'comparison_image_{i+1}.png', dpi=300, bbox_inches='tight')
            plt.show()

    # Calculate overall statistics
    base_psnrs = [r['base_psnr'] for r in results]
    att_psnrs = [r['att_psnr'] for r in results]
    improvements = [r['improvement'] for r in results]

    avg_base = np.mean(base_psnrs)
    avg_att = np.mean(att_psnrs)
    avg_imp = np.mean(improvements)

    print("\nOverall Results on Set14:")
    print(f"Average Base RCAN PSNR: {avg_base:.2f} dB")
    print(f"Average Attention RCAN PSNR: {avg_att:.2f} dB")
    print(f"Average Improvement: {avg_imp:.2f} dB")

    # Statistical significance test
    import scipy.stats as stats
    t_stat, p_value = stats.ttest_rel(att_psnrs, base_psnrs)
    print(f"\nStatistical Analysis:")
    print(f"T-statistic: {t_stat:.4f}")
    print(f"P-value: {p_value:.4f}")

    # Save detailed results to file
    with open('evaluation_results.txt', 'w') as f:
        f.write("Set14 Evaluation Results\n")
        f.write("=======================\n\n")
        f.write(f"Average Base RCAN PSNR: {avg_base:.2f} dB\n")
        f.write(f"Average Attention RCAN PSNR: {avg_att:.2f} dB\n")
        f.write(f"Average Improvement: {avg_imp:.2f} dB\n\n")
        f.write("Per-Image Results:\n")
        f.write("----------------\n")
        for r in results:
            f.write(f"\nImage {r['image_idx']}:\n")
            f.write(f"Base PSNR: {r['base_psnr']:.2f} dB\n")
            f.write(f"Attention PSNR: {r['att_psnr']:.2f} dB\n")
            f.write(f"Improvement: {r['improvement']:.2f} dB\n")
            f.write("Attention Stats:\n")
            for k, v in r['attention_stats'].items():
                f.write(f"  {k}: {v:.4f}\n")

    return results

if __name__ == "__main__":
    results = evaluate_for_paper()


Solid results and statistically significant. Many more refinements to try out from here, but we're at the deadline.